#Questandachievement Automata Industry Text To Image Tool
> Nothing Special Here, go look other tool that is better

## Program Configuration

In [ ]:
LoggingLevel="INFO"
#HyperParameters Configuration for The (Most of the time this doesn't need to be changed)
num_samples = 4
dynamic_guidance=1
guidance=11 #a number or weight of discipline towards the prompt that we input
steps=50 # predefined steps : just Generation Steps ( Too much steps will cause a confusing Generated Image, Too Small steps cause a strange unstable generated image) the best after being rigorously tested is 27 sample
dynamic_steps=1 #override predefined steps to a more randomized one
#prompt is manually input by the users through the UI
# 18:6 aspect ratio
# use this as fallback
width_Ratio=16
height_Ratio=9
safe_memory_offset=1420 # safe memory offset megabyte just in case if the usage fluctuate beyond the expected constant multiplier
enable_experimental_memory_management=1 # allows to save immense amount of memory allows to enable quality boost internal resolution without memory allocation failures (which theoretically allows 2GB shitty vram GPU able run this program)
enable_experimental_quality_boost=0 # this enables for more details my bumping the internal resolution when rendering. if and only if you enable experimental memory management it will work
#originally this was 0.029 however it was brought down to 0.023 because of the memory management improvement, then because there is memory management improvement it brought down to  0.015
# and the newer version ReEngineered v0.1 seems to be running under 0.01 successfully
#megsPixelConstantDiffusion=0.0038
megsPixelConstantDiffusion=0.0068 # in Megabytes/Pixel unit Obtained by using controlled experiment with the CUDA Device 15GB with 11.9 GB with the amount of pixel 499968
#maximumPixelQuota=1140000 # I Accidentally discovered that if you allocated too much Quota the model will errors out and its erroring about its model weight, I have a feeling that its erroring about we acessing out of bound matrix elements weight. And having a higher resolution also have a side effect on the accuracy with the result from the prompt that we requested, i think its better to implement higher resolution to upscaler.
"""
Note for developer: 262144px quota first pass, then use the full maximum pixel quota for maximum detail and stability
I heard from Automatic1111 that these kind of models were trained using 512*512 which is 262144px at most, so maybe as for multicycle optimization we limit ourself to 262144 pixel then we bump it up to the maximum pixel quota
"""
trainedModelPxQuota=262144
#trainedModelPxQuota=200144
maximumPixelQuota=2913000 #Using the experimental Enhanced L1 Wrapper the maximum pixel quota unlocked to a more higher than 1119000px (Try higher than 2000000)
tileMax1by1=256 #px 1:1 tiling maximum resolution square
"""
Based on observation of one sample gsdf/counterfeit-v2.5 Max pixel that can be pushed before it completely gave up is ~2913000px which is way better than 1130000px
"""

# this is constant for using vae tiling diffusion
globalStaticSeed=694200  # predefined seed : seed is something define on what kind of image that is generated
randomize_seed=1 #override predefined seed
style_strength=0.27 # No this is not about Smokin Sexy Style Strength
style_strength_img2img=0.127 #Conceptually, indicates how much to transform the reference image, keep this at low number
predefinedModelTarget="gsdf/Counterfeit-V2.5"
predefinedModelUpscalerTarget="stabilityai/sd-x2-latent-upscaler" #this no longer used
objectSampleEmbeddingFolder="./objectSampleTrainingEmbed"
experimental_factorQualityBoost=2 # if and only if experimental_quality_boost is enabled
# because of 992x504 struggles to operate on 15GB we can assume or make assumption to create a constant (15000-(safe Memory offset[1200])/(992*504)) = 0.03 Megs/Pixel
# with 0.03 Megs/pixel we can calculate maximum free Memory that can be used. for instance if the free memory is 8 GB we can assume 8096 MB then calculate the maximum theoretical pixel that can be rendered
# ((8096-safe_memory_offset)/megsPixelConstantDiffusion) we got (8096-1024)/0.03 = 235733.33333333333333333333 or round it to 235733
# then we divide it into the Width and height ratio for instance its 16:9 width_Ratio=16 and height_Ratio=9
# width_Internal = ((TotalPixel*width_ratio)/height_ratio)^(1/2) = ((235733*16)/9)^(1/2) = 647.3645718518189723139 # but since it has to be multiplier of 8 thus we need to round it first which is 647 then modulo it to found the nearest 8 multiplier (Example on my calculator round(647.3645718518189723139)+ mod(round(647.3645718518189723139); 8) = 654 Pixel width)
# thus we can conclude the width_Internal Formula is round(((TotalPixel*width_ratio)/height_ratio)^(1/2))-mod(round(((TotalPixel*width_ratio)/height_ratio)^(1/2)); 8) = width_Internal

# height_Internal = round((((TotalPixel-width_Internal)*9)/16)^(1/2))-mod(round((((TotalPixel-width_Internal)*9)/16)^(1/2)); 8)
# height_Internal = round((((235733-width_Internal)*9/16))^(1/2))-mod(round(((235733-width_Internal)*9/16)^(1/2)); 8) = 360p
# thus we got 654x360p for its rendering Resolution

# To get a higher Resolution adjust the resolution below after upscaling ofc
# Target 4K because Homu request everything in 4k xD "no 4k no life xd"

#ah sorry homu i cant do 4k only 3240 is possible
#4k cause the program to crash?

maxTokenSegmentPerSubmission=15 #maximum Segment per submission (Increasing this hyperparameter might increase the contextual awareness of the program, however putting this param too high might cross the 77 token boundaries of CLIP)
maxToken=27 #maximum token inide the tokenSegmentFocus (easy explanation : the maximum number of words on each commas before being merged)
watermarkStyle=1
#NOTE : ratioOverride every prompt result might be different so do a trial and error on how it fits
#ratioOverride=[5.51333333333333333333, 8.78] #A4 Paper Portrait #Resolution Ratio Override
#you can write the ratioOverride before submitting the prompt

enforceEnableCheckOverride = True #I don't want to explain on what this is all about, but what i can tell you is that if you remove this or put this to False it will cause generation failure (´･ᴗ･ ` )


In [ ]:
# Quality Cheating (Allows user to just input a more simplistic input but still have a really good output of the model)
# Prompt Injection HyperParameters
# this has too many Negative Prompt which causes the quality to degrade
#neg_prompt = "low_quality, extra_fingers, extra_legs, extra limbs, non_detailed_body, broken_teeth, bad_composition, non_detailed_face, non_detailed_environment, EasyNegative, lowres, blurry, inconsistent_face_shape, inconsistent_reflection, morphing_hand, merged_limbs, merged_buildings, merged_object, inconsistent_eye_reflection, unleveled_object, inconsistent_face_outline, different_color_eyes, textureless_object, textureless_surface, non_shining_skin, inconsistent_circle disconnected line, inconsistent_background, non_optimized_portrait, low quality environment, unrealistic entity duplication, merged limbs, disproportional eyes, disproprotional_limbs, not_masterpiece, not_the_best_quality, low_quality, low_face_quality, strange_teeth, weird_hand_gesture, Wrong_body_posture, strange_orientation, ugly, face_doesnt_render_properly, no_dirt_lens, no_smudge_lens, weird_body_proportion, Weird_painting_proportion, missing_lensflare, missing_motionblur, missing_legs, missing_hands ,bad_anatomy, error_body, error_eyes, error_cloth, not_well_detailed_cloth, error_hair, error_arm, error_hands, not_well_detailed, bad_hands, error_fingers, bad_fingers, missing_fingers, error_legs, bad_legs, multiple_legs, missing_legs, error_lighting, error_shadow, error_reflection, text, error, extra_digit, fewer_digits, cropped, worst_quality, low_quality, normal_quality, jpeg_artifacts, signature, watermark, username, blurry"
#neg_prompt = "low_quality, frame, watermark, stretched hands, doubled_object, deformed_face, deformed_eyes, SDR, wrong_gender, inconsistent_context_between_images, inconsistent_detail_with_input_images, strange_limbs_posture, strange_hands_posture, strange_legs_posture, Standard_Definition_Range_image, missing_foot, missing_hands, extra_fingers, extra_legs, extra_limbs, non_detailed_body, broken_teeth, bad_composition, non_detailed_face, non_detailed_environment, EasyNegative, lowres, blurry, inconsistent_face_shape, inconsistent_reflection, morphing_hand, merged_limbs, merged_buildings, merged_object, inconsistent_eye_reflection, unleveled_object, inconsistent_face_outline, different_color_eyes, textureless_object, textureless_surface, non_shining_skin, inconsistent_circle disconnected line, inconsistent_background, non_optimized_portrait, low quality environment, unrealistic entity duplication, merged limbs, disproportional eyes, disproprotional_limbs, not_masterpiece, not_the_best_quality, low_quality, low_face_quality, strange_teeth, weird_hand_gesture, Wrong_body_posture, strange_orientation, ugly, ugly face, more than two hands, more than two legs, ugly body, ugly limbs, face_doesnt_render_properly, no_dirt_lens, no_smudge_lens, weird_body_proportion, Weird_painting_proportion, missing_lensflare, missing_motionblur, missing_legs, missing_hands ,bad_anatomy, error_body, error_eyes, error_cloth, not_well_detailed_cloth, error_hair, error_arm, error_hands, not_well_detailed, bad_hands, error_fingers, bad_fingers, missing_fingers, error_legs, bad_legs, multiple_legs, missing_legs, error_lighting, error_shadow, error_reflection, text, error, extra_digit, fewer_digits, cropped, worst_quality, low_quality, normal_quality, jpeg_artifacts, signature, watermark, username, lost context from image, (by bad-artist:1.8), (mutated hands and fingers:1.8), (Too many hands:1.8), (long body:1.9), (Mutation, poorly drawn:1.4), (lowres:1.8), (low quality:1.8), out of frame, (out_of_frame:1.9), inconsistent_eyes, inconsistent_eyes_reflection , (poorly drawn eyes:1.7, bad detail eyes:1.8), unlogical place, disconnected_object, disconnected_structure, inconsistent logic, poorly drawn face, bad anatomy, malformed anatomy, malformed limbs, missing limbs, missing objects detail, merged limbs, (blurry:1.7), inconsistent straight line, cropped"
neg_prompt = "low quality, extra fingers, extra legs, extra limbs, non_detailed_body, broken_teeth, bad_composition, non_detailed_face, non_detailed_environment, EasyNegative, lowres, blurry, inconsistent_face_shape, inconsistent_reflection, morphing_hand, merged_limbs, merged_buildings, merged_object, inconsistent_eye_reflection, unleveled_object, inconsistent_face_outline, different_color_eyes, textureless_object, textureless_surface, non_shining_skin, inconsistent_circle disconnected line, inconsistent_background, non_optimized_portrait, low quality environment, unrealistic entity duplication, merged limbs, disproportional eyes, disproprotional_limbs, not_masterpiece, not_the_best_quality, low_quality, low_face_quality, strange_teeth, weird_hand_gesture, Wrong_body_posture, strange_orientation, ugly, face_doesnt_render_properly, no_dirt_lens, no_smudge_lens, weird_body_proportion, Weird_painting_proportion, missing_lensflare, missing_motionblur, missing_legs, missing_hands ,bad_anatomy, error_body, error_eyes, error_cloth, not_well_detailed_cloth, error_hair, error_arm, error_hands, not_well_detailed, bad_hands, error_fingers, bad_fingers, missing_fingers, error_legs, bad_legs, multiple_legs, missing_legs, error_lighting, error_shadow, error_reflection, text, error, extra_digit, fewer_digits, cropped, worst_quality, low_quality, normal_quality, jpeg_artifacts, signature, watermark, username, blurry"
#fixed version
#neg_prompt keep the quality at default at least standard
paintingStyle="Anime" #additional Default Style for the Painting to the Prompt
qualityEnforcement="masterpiece, best_quality, HDR"
specialFX="sparkle, lensflare, viewfinder" #additional Default SpecialFX for the Painting to the Prompt


## Define the Required Methods and/or functions



#### Presistent *Required* storage

##### F1.8 TTF

In [ ]:
rawDataF18 = "..."

##### Free3of9 font

In [ ]:
rawDataFree3of9 = "AAEAAAAKAIAAAwAgT1MvMnu0fOcAAACsAAAATmNtYXAB+wLuAAAA/AAAAXJnbHlmfm8KBAAAAnAAAA22aGVhZBh23U8AABAoAAAANmhoZWEI4QNPAAAQYAAAACRobXR4AxAAAAAAEIQAAABgbG9jYUyKT/gAABDkAAAAYG1heHAANQAWAAARRAAAACBuYW1lSHvdPQAAEWQAAAJxcG9zdAADAAAAABPYAAAAJAAAAxABkAAFAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACQAAAAAAAAAAAAAAAAAAAAAAAAAAAABzZnQAAAAAIABfBgQAAAAABgQAAAAAAAAAAgABAAAAAAAUAAMAAQAAARoAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAWAAAAA4ACAACAAYAIAAlAC8AOQBaAF///wAAACAAJAAqADAAQQBf//8ACgAAAAD/1P/N/8sAAQAAAAwADgAAAAAAAAAAACsALgADAC0AAAAoACkALAAAAAEAAAAAADQGAAADAAAzIxEzNDQ0BgD3AAEAAAAAAIwGAAADAAAzIxEzjIyMBgAAAAUAAAAAAtwGAAADAAcACwAPABMAADMjETMTIxEzASMRMwEjETMTIxEz9DQ0wIyM/oA0NAJAjIxoNDQGAPoABgD6AAYA+gAGAPoABgAAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMBIxEzAyMRMwEjETMDIxEzAtw0NP3ANDRoNDQCQIyMwIyMBgD6AAYA+gAGAPoABgD6AAYAAAUAAAAAAtwGAAADAAcACwAPABMAADMjETMBIxEzASMRMwEjETMDIxEz9DQ0AeiMjP2wjIwBkDQ0aDQ0BgD6AAYA+gAGAPoABgD6AAYAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMBIxEzEyMRMwEjETMDIxEzAbQ0NP6ANDTAjIwB6IyMwDQ0BgD6AAYA+gAGAPoABgD6AAYAAAUAAAAAAtwGBAADAAcACwAPABMAACEjETMTIxEzASMRMwEjETMDIxEzAUyMjMA0NP6AjIwCUDQ0aDQ0BgD6AAYA+gAGAPoEBgD5/AYAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMBIxEzAyMRMwEjETMBIxEzAhw0NP6ANDRoNDQCqIyM/tiMjAYA+gAGAPoABgD6AAYA+gAGAAAABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwEjETMBIxEzASMRMwEjETMCDIyM/ug0NAGANDT+GIyMAlA0NAYA+gAGAPoABgD6AAYA+gAGAAAFAAAAAALcBgAAAwAHAAsADwATAAAhIxEzASMRMxMjETMBIxEzEyMRMwJ0NDT9wDQ0wIyMARiMjNA0NAYA+gAGAPoABgD6AAYA+gAGAAAFAAAAAALcBgAAAwAHAAsADwATAAAhIxEzAyMRMwMjETMBIxEzEyMRMwFcNDTANDRoNDQB6IyMwIyMBgD6AAYA+gAGAPoABgD6AAYAAAAFAAAAAALcBgAAAwAHAAsADwATAAAhIxEzAyMRMwEjETMBIxEzASMRMwJ0jIzANDT+2IyMAlA0NP4YNDQGAPoABgD6AAYA+gAGAPoABgACAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMBIxEzEyMRMwEjETMTIxEzAbQ0NP6ANDTAjIwBgIyMaDQ0BgD6AAYA+gAGAPoABgD6AAYAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMDIxEzASMRMxMjETMDIxEzAtyMjMA0NP7YNDRoNDTQjIwGAPoABgD6AAYA+gAGAPoABgAAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMBIxEzASMRMwEjETMBIxEzAVw0NP7YNDQCqIyM/hiMjAEoNDQGAPoABgD6AAYA+gAGAPoABgAABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwEjETMBIxEzAyMRMwEjETMCdDQ0/hiMjAEoNDRojIwBkDQ0BgD6AAYA+gAGAPoABgD6AAYAAAAFAAAAAALcBgAAAwAHAAsADwATAAAhIxEzASMRMwMjETMBIxEzASMRMwIcNDT+gDQ0aDQ0ASiMjAGAjIwGAPoABgD6AAYA+gAGAPoABgABAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMBIxEzASMRMwEjETMBIxEzAbSMjAEoNDT+GDQ0AYA0NP4YjIwGAPoABgD6AAYA+gAGAPoABgAABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwEjETMTIxEzEyMRMwEjETMCdDQ0/cA0NMCMjMCMjAEoNDQGAPoABgD6AAYA+gAGAPoABgAABQAAAAAC3AYAAAMABwALAA8AEwAAMyMRMxMjETMDIxEzASMRMwMjETOcNDRoNDTQNDQCqIyMwIyMBgD6AAYA+gAGAPoABgD6AAYAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMTIxEzASMRMxMjETMDIxEzAnSMjGg0NP4YNDRoNDTQjIwGAPoABgD6AAYA+gAGAPoABgABAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMBIxEzASMRMxMjETMBIxEzAVw0NP7YNDQCQIyMaDQ0/hiMjAYA+gAGAPoABgD6AAYA+gAGAAAABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwEjETMDIxEzASMRMwEjETMC3DQ0/cA0NGg0NAEojIwBGIyMBgD6AAYA+gAGAPoABgD6AAYAAAAFAAAAAALcBgAAAwAHAAsADwATAAAhIxEzASMRMwMjETMTIxEzAyMRMwLcjIz+6DQ00DQ0aDQ00IyMBgD6AAYA+gAGAPoABgD6AAYAAwAFAAAAAALcBgAAAwAHAAsADwATAAAhIxEzASMRMwEjETMBIxEzAyMRMwFcNDT+2DQ0AqiMjP7oNDTQjIwGAPoABgD6AAYA+gAGAPoABgAQAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMBIxEzASMRMwMjETMTIxEzAtw0NP2wjIwBkDQ00IyMaDQ0BgD6AAYA+gAGAPoABgD6AAYAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMBIxEzAyMRMwEjETMBIxEzAcQ0NP7YNDRoNDQCqIyM/oCMjAYA+gAGAPoABgD6AAYA+gAGAMQABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwEjETMDIxEzASMRMwEjETMCHDQ0/tg0NGiMjAJQNDT+2IyMBgD6AAYA+gAGAPoABgD6AAYAAAAFAAAAAALcBgAAAwAHAAsADwATAAAhIxEzASMRMxMjETMBIxEzASMRMwIcNDT+GDQ0wIyMAeg0NP7YjIwGAPoABgD6AAYA+gAGAPoABgAAAAUAAAAAAtwGAAADAAcACwAPABMAADMjETMTIxEzAyMRMwEjETMBIxEznDQ0aDQ00DQ0AZCMjAEYjIwGAPoABgD6AAYA+gAGAPoABgAAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMTIxEzASMRMxMjETMDIxEzAhyMjMA0NP4YNDRoNDTQjIwGAPoABgD6AAYA+gAGAPoABgAAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMBIxEzASMRMxMjETMBIxEzAVw0NP7YNDQB6IyMwDQ0/hiMjAYA+gAGAPoABgD6AAYA+gAGAAAABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwEjETMDIxEzASMRMxMjETMC3DQ0/cA0NGg0NAEojIzAjIwGAPoABgD6AAYA+gAGAPoABgAABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwEjETMTIxEzEyMRMxMjETMC3IyM/bCMjMA0NGg0NGg0NAYA+gAGAPoABgD6AAYA+gAGAAUABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwEjETMTIxEzEyMRMwEjETMC3IyM/nCMjGg0NGg0NP4YNDQGAPoABgD6AAYA+gAGAPoABgAABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwEjETMBIxEzEyMRMwMjETMBpIyM/uiMjAGANDTQNDRoNDQGAPoABgD6AAYA+gAGAPoABgAABQAAAAAC3AYAAAMABwALAA8AEwAAMyMRMwEjETMDIxEzASMRMwEjETP0NDQB6IyMwDQ0/hg0NAGAjIwGAPoABgD6AAYA+gAGAPoABgAABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwEjETMTIxEzASMRMxMjETMCDIyM/oCMjMA0NAEoNDRoNDQGAPoABgD6AAYA+gAGAPoABgAABQAAAAAC3AYAAAMABwALAA8AEwAAISMRMwMjETMDIxEzASMRMwEjETMC3DQ00IyMwIyMASg0NP3ANDQGAPoABgD6AAYA+gAGAPoABgAABQAAAAAC3AYAAAMABwALAA8AEwAAMyMRMwEjETMBIxEzASMRMwEjETP0NDQB6IyM/oA0NP7YNDQB6IyMBgD6AAYA+gAGAPoABgD6AAYAAAAFAAAAAALcBgAAAwAHAAsADwATAAAhIxEzASMRMwEjETMBIxEzAyMRMwJ0jIz+GIyMAlA0NP7YNDRoNDQGAPoABgD6AAYA+gAGAPoABgAAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMTIxEzASMRMwEjETMBIxEzAbQ0NMCMjP7YjIwBkDQ0/Vg0NAYA+gAGAPoABgD6AAYA+gAGAAAABQAAAAAC3AYAAAMABwALAA8AEwAAMyMRMxMjETMTIxEzEyMRMwEjETP0NDTANDTANDRoNDT9WDQ0BgD6AAYA+gAGAPoABgD6AAYAAAUAAAAAAtwGAAADAAcACwAPABMAADMjETMTIxEzASMRMwMjETMBIxEz9DQ0wDQ0ASg0NMA0NP4YNDQGAPoABgD6AAYA+gAGAPoABgAGAAUAAAAAAtwGAAADAAcACwAPABMAADMjETMBIxEzAyMRMwEjETMBIxEz9DQ0ASg0NMA0NAGANDT9WDQ0BgD6AAYA+gAGAPoABgD6AAYAAAUAAAAAAtwGAAADAAcACwAPABMAACEjETMDIxEzASMRMxMjETMBIxEzAVw0NMA0NAGANDTANDT9WDQ0BgD6AAYA+gAGAPoABgD6AAYAAAAAAQAAAAAAADoO7dpfDzz1AAAIAAAAAAAAAAAAAAAAALaKklYAAAAAAtwGBAAAAAAAAAAAAAAAAAABAAAGBAAAAAADGgAAADQC3AAAAAAAAAAAAAAAAAAAAAAAAQMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADAAYAD8AZgCNALQA2wEDASsBUgF5AaEByAHvAhcCPwJnAo8CtgLcAwMDKwNTA3oDogPJA/EEGQRBBGgEjwS3BN4FBQUsBVMFegWhBcgF8AYYBkAGZgaNBrQG2wABAAAALwAUAAUAAAAAAAEAAAAAAAAAAAAAAAAAAAABAAAAEADGAAEAAAAAAAAACQEoAAEAAAAAAAEAAwEPAAEAAAAAAAIABwD6AAEAAAAAAAMAAwC3AAEAAAAAAAQAAwCOAAEAAAAAAAUADABqAAEAAAAAAAYAAwBXAAEAAAAAAAcACQAAAAMAAQQJAAAAbgExAAMAAQQJAAEAFgESAAMAAQQJAAIADgEBAAMAAQQJAAMAQAC6AAMAAQQJAAQAJgCRAAMAAQQJAAUAGAB2AAMAAQQJAAYAEABaAAMAAQQJAAcATgAJVHJhZGVtYXJrAEYAcgBlAGUAIAAzACAAbwBmACAAOQCZACAAVAByAGEAZABlAG0AYQByAGsAIABvAGYAIABNAGEAdAB0AGgAZQB3ACAAVwBlAGwAYwBoTmV3AEYAcgBlAGUAMwBvAGYAOVZlcnNpb24gMS4wMABWAGUAcgBzAGkAbwBuACAAMQAuADAAMU5ldwBGAHIAZQBlACAAMwAgAG8AZgAgADkAIABSAGUAZwB1AGwAYQByTmV3AEYAcgBlAGUAIAAzACAAbwBmACAAOQAgAFIAZQBnAHUAbABhAHIAOgBWAGUAcgBzAGkAbwBuACAAMQAuADAAMVJlZ3VsYXIAUgBlAGcAdQBsAGEAck5ldwBGAHIAZQBlACAAMwAgAG8AZgAgADlDb3B5cmlnaHQAQwBvAHAAeQByAGkAZwBoAHQAIACpACAAMQA5ADkANwAgAGIAeQAgAE0AYQB0AHQAaABlAHcAIABXAGUAbABjAGgALgAgAEEAbABsACAAUgBpAGcAaAB0AHMAIABSAGUAcwBlAHIAdgBlAGQALgAAAAAAAAAAAAAAAAAAAAADAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA=="

##### Riesling Font

In [ ]:
rawDataRiesling = "..."

##### q7Logo


In [ ]:
q7LogoEnc = "..."

##### Merch Generation Assets

In [ ]:
TshirtTemplateRaw = "..."

##### Textual_Inversion Training Script (Embedding)


In [ ]:
textual_inversion_script="..."

### Req **Method** and **Assets Extraction**

In [ ]:
versionProgram="dE_QARE_0.6.5_Beta"

In [ ]:
#Module Management Function Section
#import essential library
import os
import re
import psutil
import gc
import sys
import random
import glob
import logging
import importlib as imp
import pip
import multiprocessing
import platform
from datetime import datetime
import datetime as dt
import base64

def installReqModules():
  pip.main(['install', 'opencv-python'])
  pip.main(['install', 'memory_profiler'])
  pip.main(['install', 'nvidia-pyindex'])
  pip.main(['install', 'nvidia-tensorrt'])
  pip.main(['install', 'wordsegment'])
  pip.main(['install', 'speedtorch'])
  pip.main(['install', 'transformers'])
  pip.main(['install', 'scipy'])
  pip.main(['install', 'ftfy'])
  pip.main(['install', 'ipywidgets'])
  pip.main(['install', 'datasets'])
  pip.main(['install', 'pynvml'])
  pip.main(['install', 'matplotlib'])
  pip.main(['install', 'Pillow'])
  pip.main(['install', 'accelerate'])
  pip.main(['install', 'diffusers'])
  pip.main(['install', 'invisible-watermark'])
  pip.main(['install', 'pillow-heif'])
  pip.main(['install', 'xformers'])
  pip.main(['install', 'triton'])
  pip.main(['install', 'nicegui'])
  pip.main(['install', 'progressbar'])
  pip.main(['install', 'exif'])
  pip.main(['install', 'git+https://github.com/sberbank-ai/Real-ESRGAN.git'])
  #pip.main(['install', 'smc'])
  #pip.main(['install', 'smc.freeimage'])
  pip.main(['install', 'urllib'])
  #if 'ubuntu' in platform.version().lower():
  if "linux" in sys.platform():
    os.system("sudo apt install libmagickwand-dev -y")


#Import all Library
def loadModules():
  global image
  global pillow_heif
  global register_heif_opener
  global profile
  global np
  global torch
  global RealESRGAN
  global pynvml
  global autocast
  global StableDiffusionLatentUpscalePipeline
  global StableDiffusionPipeline
  global StableDiffusionImg2ImgPipeline
  global DPMSolverMultistepScheduler
  global tracemalloc
  global imgexif
  #global pgm
  global cv2
  global Image_imgMagick
  global ImagePostProcessWorker
  global ImageFont
  global ImageDraw
  global DDIMScheduler
  global UI
  global image2
  global wget
  global GPT4All
  global progressbar
  global imageops
  global Image
  global Markdown
  global hf_hub_download
  global plt
  global StableDiffusionInstructPix2PixPipeline
  global EulerAncestralDiscreteScheduler
  global StableDiffusionXLPipeline
  global StableDiffusionXLImg2ImgPipeline
  global StableDiffusionXLInpaintPipeline

  numNotInstalled=0
  #https://stackoverflow.com/questions/17271444/how-to-install-a-missing-python-package-from-inside-the-script-that-needs-it

  try:
    from IPython.display import Markdown
    from huggingface_hub import hf_hub_download
  except ImportError:
    installReqModules()
    from IPython.display import Markdown
    from huggingface_hub import hf_hub_download

  try:
    from matplotlib import pyplot as plt
  except ImportError:
    installReqModules()
    from matplotlib import pyplot as plt

  try:
    import progressbar
  except ImportError:
    installReqModules()
    import progressbar

  try:
    import cv2
  except ImportError:
    installReqModules()
    import cv2

  try:
    from urllib import request as wget
  except ImportError:
    installReqModules()
    from urllib import request as wget

  try:
    from nicegui import ui as UI
  except ImportError:
    installReqModules()
    from nicegui import ui as UI

  try:
    from PIL import Image
    from PIL import Image as image
    from PIL import ImageOps as imageops
    from PIL import Image as image2
    from PIL import ImageEnhance as ImagePostProcessWorker
    from PIL import ImageFont, ImageDraw
  except ImportError:
    installReqModules()
    from PIL import Image
    from PIL import Image as image
    from PIL import ImageOps as imageops
    from PIL import Image as image2
    from PIL import ImageEnhance as ImagePostProcessWorker
    from PIL import ImageFont, ImageDraw
  #import pgmagick as pgm



  try:
    from memory_profiler import profile #memory profiler
  except ImportError:
    installReqModules()
    from memory_profiler import profile #memory profiler

  try:
    import numpy as np
  except ImportError:
    installReqModules()
    import numpy as np

  try:
    import torch
    from torch import autocast
  except ImportError:
    installReqModules()
    import torch
    from torch import autocast

  try:
    from RealESRGAN import RealESRGAN
  except ImportError:
    installReqModules()
    from RealESRGAN import RealESRGAN

  try:
    import pynvml
  except ImportError:
    installReqModules()
    import pynvml

  try:
    import transformers
  except ImportError:
    installReqModules()
    import transformers

  try:
    import accelerate
  except ImportError:
    installReqModules()
    import accelerate

  try:
    import tensorrt
  except ImportError:
    installReqModules()
    import tensorrt

  try:
    from diffusers import StableDiffusionLatentUpscalePipeline
    from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler, DDIMScheduler, StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler
    from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline, StableDiffusionXLInpaintPipeline
  except ImportError:
    installReqModules()
    from diffusers import StableDiffusionLatentUpscalePipeline
    from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler, DDIMScheduler, StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler
    from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline, StableDiffusionXLInpaintPipeline

  try:
    import tracemalloc
  except ImportError:
    installReqModules()
    import tracemalloc

  try:
    from exif import Image as imgexif
  except ImportError:
    installReqModules()
    from exif import Image as imgexif

  try:
    import ipywidgets as widgets
  except ImportError:
    installReqModules()
    import ipywidgets as widgets

  try:
    import array
  except ImportError:
    installReqModules()
    import array

  try:
    import pillow_heif
    from pillow_heif import register_heif_opener
  except ImportError:
    installReqModules()
    import pillow_heif
    from pillow_heif import register_heif_opener

class MyProgressBar(): #https://stackoverflow.com/questions/37748105/how-to-use-progressbar-module-with-urlretrieve
    def __init__(self):
        self.pbar = None

    def __call__(self, block_num, block_size, total_size):
        if not self.pbar:
            self.pbar=progressbar.ProgressBar(maxval=total_size)
            self.pbar.start()

        downloaded = block_num * block_size
        if downloaded < total_size:
            self.pbar.update(downloaded)
        else:
            self.pbar.finish()

def clearModuleState():
  #https://www.oreilly.com/library/view/python-cookbook/0596001673/ch14s02.html
  #sys.modules.clear()
  #print("Function is not implemented")
  pass
  modulesList=["diffusers", "PIL", "pillow_heif", "memory_profiler", "numpy", "torch", "RealESRGAN", "pynvml", "torch", "tracemalloc", "exif"]
  #for module in modulesList:
  #  imp.reload(module)



In [ ]:
def init_shmem():
  import multiprocessing as mpshmem
  #Initialize Shared Memory if Function ever need it
  global shared_memory
  #multiprocessing.set_start_method('spawn') # if its not set to spawn there will be some CUDA error

  #allocate variable for shared multiprocessing variable (use dictionary) so we can pass along any variable from inside of the branch process
  shared_memory = mpshmem.Manager().dict()


In [ ]:
# Extract or decode the fonts into the main working directory
# https://www.codespeedy.com/convert-base64-string-to-image-in-python/
# https://www.geeksforgeeks.org/python-write-bytes-to-file/
# there are 3 files need to be decoded
# rawDataF18 rawDataFree3of9 rawDataRiesling

class specialProjectDaredevil(): #a daredevil project to an automated bussiness
  def merchSpriteAssetsExtraction():
    global TShirtTemplateImage
    TShirtDecoded = base64.b64decode((TshirtTemplateRaw))
    with open("fullFrontRearTShirtTemplate.jpg", "wb") as binary_file:
      binary_file.write(TShirtDecoded)

def requestSprites():
  global WatermarkPNGImprinted_q7
  q7Logo = base64.b64decode((q7LogoEnc))
  with open("q7Logo.png", "wb") as binary_file:
    binary_file.write(q7Logo)
  WatermarkPNGImprinted_q7 = image.open("./q7Logo.png").convert('RGBA')

def requestRequiredExtraScript():
  #textual_inversion_script
  textual_inversion_script_decode = base64.b64decode((textual_inversion_script))
  with open("textual_inversion.py", "wb") as binary_file:
    binary_file.write(textual_inversion_script_decode)
  print("textual_inversion.py Written!")
  if not os.path.exists(objectSampleEmbeddingFolder):
    print("Preparing Folders")
    os.makedirs(objectSampleEmbeddingFolder)
  else:
    pass

def requestFont():
  F18Font = base64.b64decode((rawDataF18))
  with open("f18.otf", "wb") as binary_file:
    binary_file.write(F18Font)

  Free3of9Font = base64.b64decode((rawDataFree3of9))
  with open("free3of9.ttf", "wb") as binary_file:
    binary_file.write(Free3of9Font)

  RieslingFont = base64.b64decode((rawDataRiesling))
  with open("riesling.ttf", "wb") as binary_file:
    binary_file.write(RieslingFont)

In [ ]:

#Main Program Function Section

#some modification was based on https://qiita.com/cress_cc/items/6012ecfabb33411a3d9b ( Sorry its a japanese website :/ )

logging.basicConfig(level=LoggingLevel)

#tracemalloc.start()

#Model Management
#loads Model to memory and other dependencies to generate some pixels

def loadModel2Memory_Text2Img(modelTarget, modelUpscalerTarget):
  checkModelCacheAndFreeUp() #this allows to dynamically allocate and unallocate or unload if you need to switch between Text2Image or Image2Image without filling the Memory too much
  global model_id
  global scheduler
  global model_id_upsc
  global upscaler
  model_id = selectedModel #mai
  scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, algorithm_type="dpmsolver++", subfolder="scheduler" ) # num_train_timesteps is the amount of sample that it will do (more sample which mean less noise and more quality sample) same as blender CycleX engine
  #scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
  #scheduler = DDIMScheduler()
  #model_id_upsc=modelUpscalerTarget
  #upscaler = StableDiffusionLatentUpscalePipeline.from_pretrained(model_id_upsc) #upscaler pipeline
  #Main Pipeline Text2Image
  global pipe
  pipe = StableDiffusionPipeline.from_pretrained(
  model_id,
  torch_dtype=torch_dtype,
  scheduler=scheduler)
  pipe.enable_attention_slicing()
  #pipe.enable_vae_slicing()
  #pipe.vae.enable_tiling() #not available yet
  if device == "cuda":
    pipe=pipe.to("cuda")
    pipe.vae.enable_xformers_memory_efficient_attention(attention_op=None)
    pipe.enable_xformers_memory_efficient_attention()
    #upscaler=upscaler.to("cuda")
  #upscaler.enable_attention_slicing()
  #upscaler.vae.enable_xformers_memory_efficient_attention(attention_op=None)
  if extremeMemoryPreservation == 1:
    pipe_i2i.enable_sequential_cpu_offload()
  pipe.unet.to(memory_format=torch.channels_last)
  #If you have low GPU RAM available, make sure to add a pipe.enable_attention_slicing() after sending it to cuda for less VRAM usage (to the cost of speed)
  #https://huggingface.co/stabilityai/stable-diffusion-x4-upscaler/blob/main/README.md
  if parameterSafetyCheckerModifiable == True : #blacklist model from disabled safetyChecker
    #pipe.safety_checker = lambda images, clip_input: (images, False) # disabling false positives
    pipe.safety_checker = lambda images, **kwargs: (images, [False] * len(images)) #https://www.reddit.com/r/StableDiffusion/comments/wxba44/disable_hugging_face_nsfw_filter_in_three_step/

def loadModel2Memory_Img2Img(modelTarget, modelUpscalerTarget):
  checkModelCacheAndFreeUp() #this allows to dynamically allocate and unallocate or unload if you need to switch between Text2Image or Image2Image without filling the Memory too much
  global model_id
  global scheduler
  global model_id_upsc
  global upscaler
  model_id = selectedModel #main
  scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, algorithm_type="dpmsolver++", subfolder="scheduler" ) # num_train_timesteps is the amount of sample that it will do (more sample which mean less noise and more quality sample) same as blender CycleX engine
  #scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
  #model_id_upsc=modelUpscalerTarget
  #upscaler = StableDiffusionLatentUpscalePipeline.from_pretrained(model_id_upsc) #upscaler pipeline
  #Main Pipeline Text2Image
  global pipe_i2i
  #create a new handler for pix2pix Pipeline which usually only used when there's propmpt where forcePix2PixMode invoke thus we can just assume that if the forceModeModel=5 then the pipeline is needed
  if forceModeModel != 5:
    #std Handler using Img2Img Pipeline
    pipe_i2i = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    scheduler=scheduler)
    pipe_i2i.enable_attention_slicing()
    #pipe_i2i.enable_vae_slicing()
    pipe_i2i.vae.enable_tiling()
    if device == "cuda":
      pipe_i2i=pipe_i2i.to("cuda")
      pipe_i2i.vae.enable_xformers_memory_efficient_attention(attention_op=None)
      pipe_i2i.enable_xformers_memory_efficient_attention()
      #upscaler=upscaler.to("cuda")
        #upscaler.enable_attention_slicing()
      #upscaler.vae.enable_xformers_memory_efficient_attention(attention_op=None)
      #https://huggingface.co/docs/diffusers/optimization/fp16
      if extremeMemoryPreservation == 1:
        pipe_i2i.enable_sequential_cpu_offload()
      pipe_i2i.unet.to(memory_format=torch.channels_last)
  else:
    print("[QA Automata Optimization Debug]: Experimental Pix2Pix Pipeline Invoked")
    pipe_i2i = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)
    if device == "cuda":
      pass
      pipe_i2i.to("cuda")
    pipe_i2i.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe_i2i.scheduler.config)
    pass
  #If you have low GPU RAM available, make sure to add a pipe.enable_attention_slicing() after sending it to cuda for less VRAM usage (to the cost of speed)
  #https://huggingface.co/stabilityai/stable-diffusion-x4-upscaler/blob/main/README.md
  if parameterSafetyCheckerModifiable == True : #blacklist model from disabled safetyChecker
    #pipe.safety_checker = lambda images, clip_input: (images, False) # disabling false positives
    pipe_i2i.safety_checker = lambda images, **kwargs: (images, [False] * len(images)) #https://www.reddit.com/r/StableDiffusion/comments/wxba44/disable_hugging_face_nsfw_filter_in_three_step/

def acceleratorAvailability():
  global torch_dtype
  global device
  global extremeMemoryPreservation
  extremeMemoryPreservation=0
  if torch.cuda.is_available():
    torch_dtype=torch.float16
    device="cuda"
    freeMemCheck = freeMemoryCheck() # only allow CUDA GPU that have 2500MiB free memory that can use this acceleration
    if freeMemCheck < 2700:
      if freeMemCheck < 2400:
        print("[QA Automata Optimization Critical Warning] : Can't fit model into the Memory! ")
        device="cpu"
        #megsPixelConstantDiffusion=0.029
        #https://discuss.pytorch.org/t/how-should-i-disable-using-cudnn-in-my-code/38053
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.enabled = False
        torch_dtype=torch.float32
      else:
        print("[QA Automata Optimization Warning] : GPU Memory Preservation Activated! ")
        extremeMemoryPreservation=1
        os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64,garbage_collection_threshold:0.9"
    else:
      torch.backends.cudnn.benchmark = True
      torch.backends.cuda.matmul.allow_tf32 = True

  else:
    #https://discuss.pytorch.org/t/how-should-i-disable-using-cudnn-in-my-code/38053
    #megsPixelConstantDiffusion=0.029
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.enabled = False
    torch_dtype=torch.float32
    #torch_dtype=torch.float16
    device="cpu"

def freeMemoryCheck():
  global memoryFreeAvailableMegs
  if device == "cuda":
    gpuIndexMemoryCheck=0
    def get_memory_free_MiB(gpu_index): #https://stackoverflow.com/questions/58216000/get-total-amount-of-free-gpu-memory-and-available-using-pytorch
      pynvml.nvmlInit()
      handle = pynvml.nvmlDeviceGetHandleByIndex(int(gpu_index))
      mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
      return mem_info.free // 1024 ** 2
    memoryFreeAvailableMegs = round(get_memory_free_MiB(gpuIndexMemoryCheck))
  else:
    # for non Nvidia Proprietary Use
    #>>> psutil.virtual_memory()
    #svmem(total=16479604736, available=7792549888, percent=52.7, used=6085033984, free=1404526592, active=8306073600, inactive=4833189888, buffers=73728, cached=8989970432, shared=1992929280, slab=821604352)
    # memory Availability located at Index [1] and its on Bytes form
    memoryFreeAvailableMegs = psutil.virtual_memory()[1]/pow(1024, 2)
  return memoryFreeAvailableMegs


def upscalerResolutionCheatingInit(height_Internal, width_Internal):
  # TODO: Define Free memory and determine what can be used on the memory usage
  currentDevfreeMem = freeMemoryCheck()
  #print(currentDevfreeMem)
  #from the research 6642MiB allows to upscale 1000p to 5000p or using x8 Upscaler
  # since the max allowed pixel that are rendered around ~1200000px while the upscaled version is ~66846720px theres the delta of ~65646720px
  # we can calculate the constant capability using the delta and the free mem if  c = 6642/65646720 = 0.00010117794156356936 MiB/px or ~0.0001 MiB/px
  # thus if we have 2048 MiB free we can just divide it 2048/0.0001 =  20480000 and this is Upscaling Quota
  # then we divide it max allowed pixel rendered Raw vs Upscaled Quota 20480000/1200000px
  global scalingTargetMultiplier
  global modelUpscaler
  # 4096MiB minimum to target 4920p
  # 2048MiB to target 2160p
  # <1800MiB to target 1080p
  #targetOutputResolution=4920
  if currentDevfreeMem >= 4096:
    targetOutputResolution=5120
  if currentDevfreeMem < 4096:
    targetOutputResolution=4920
  if currentDevfreeMem < 2048:
    targetOutputResolution=2160
  if currentDevfreeMem < 1800:
    targetOutputResolution=1080


  if width_Ratio > height_Ratio:
    scalingTargetMultiplier=round(targetOutputResolution/height_Internal)
    logging.debug("[QA Automata Optimization Debug] : Landscape Configuration", scalingTargetMultiplier)
  else:
    scalingTargetMultiplier=round(targetOutputResolution/width_Internal)
    logging.debug("[QA Automata Optimization Debug] : Portrait or Square Configuration", scalingTargetMultiplier)
  # Logic to Lock the scaling to 2x 4x or 8x
  xscaling=scalingTargetMultiplier
  xPow2=(2 ** 2) % xscaling
  #print(xPow2)
  xPow3=(2 ** 3) % xscaling
  #print(xPow3)
  if xPow2 > xPow3 or xscaling > 8:
    multPow=3 # max upscaler x8 thus 2^8
  if xPow2 == 0 or xPow3 > xPow2:
    multPow=2
  if xscaling == 2 or xscaling < 2:
    multPow=1
  scalingTargetMultiplier=2**multPow
  logging.debug("[QA Automata Optimization Debug] : Upscaling Target Multiplier", scalingTargetMultiplier)
  # RealESRGan only supported x2 x4 x8 scaling
  logging.debug("[QA Automata Optimization Debug] : x8 Model has been loaded")
  modelUpscaler = RealESRGAN(device, scale=scalingTargetMultiplier)
  if multPow == 1:
    modelUpscaler.load_weights('weights/RealESRGAN_x2.pth', download=True)
  if multPow == 2:
    modelUpscaler.load_weights('weights/RealESRGAN_x4.pth', download=True)
  if multPow == 3:
    modelUpscaler.load_weights('weights/RealESRGAN_x8.pth', download=True)
#----------------------------------------------------------------------------------
#general Memory Management function




# Global Flush
#basically flush all Model that is currently on GPU Memory if it is really required (Because by default if you tried to load other pipe for instance from t2i to i2i it will unload the other pipe memory)
def checkModelCacheAndFreeUp():
  global modelUpscaler
  global scheduler
  global pipe
  global pipe_i2i
  global upscaler
  try:
    modelUpscaler
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del modelUpscaler
  try:
    scheduler
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del scheduler
  try:
    pipe
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del pipe
  try:
    pipe_i2i
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del pipe_i2i
  try:
    upscaler
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del upscaler
  #https://stackoverflow.com/questions/55322434/how-to-clear-cuda-memory-in-pytorch
  acceleratorAvailability()
  if device == "cuda":
    torch.cuda.empty_cache()
  gc.collect()

def imageFlushMemoryStorageFlush():
  global imageLoadPhase1
  global imageLoadPhase2
  global images_postProcessed
  global imageRescaledForShitGradio
  global submittedImage
  try:
    imageLoadPhase1
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del imageLoadPhase1
  try:
    imageLoadPhase2
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del imageLoadPhase2
  try:
    images_postProcessed
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del images_postProcessed
  try:
    imageRescaledForShitGradio
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del imageRescaledForShitGradio
  try:
    submittedImage
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del submittedImage
  try:
    shared_memory
  except NameError:
    pass
  else:
    logging.debug("[QA Automata Optimization Debug] : varMem was eating ")
    del shared_memory


# Global Flush
#basically flush all Model that is currently on GPU Memory if it is really required (Because by default if you tried to load other pipe for instance from t2i to i2i it will unload the other pipe memory)
def freeUpGlobalMemory():
  checkModelCacheAndFreeUp()
  imageFlushMemoryStorageFlush()

class ReAIError(Exception):
    pass

class NLPProcessor():
  #MAIN
  def AnalyzeText_L2(prompt): #wrapped in multiprocessing module for memory management flushing reasons, for programming call this method instead of the L1 and use this method as the main launcher for the text analysis
    #global categoryResult # learn on how to use shared variables
    NLPp = multiprocessing.Process(target=NLPProcessor.AnalyzeText_L1, args=(prompt,))
    NLPp.start()
    NLPp.join()
    #return "stub","stub"
  #----
  #Suplementary Methods
  def initMainEngine():
    global gptM
    gptM = GPT4All()
    gptM.open()
    #initialization
    gptM.prompt('Create an Appropriate response from the chat')
    initGPTContextOutput = gptM.prompt('Hello there')
    print("[QA Automata Optimization Debug] : InitGPT", initGPTContextOutput)
    initGPTContextOutput = gptM.prompt('So today im going to ask you to create some short story and the story title')
    print("[QA Automata Optimization Debug] : InitGPT", initGPTContextOutput)

  def AnalyzeText_L1(prompt): #not wrapped in multiprocessing
  #WARNING : Since it is running wrapped inside of the L2 method or wrapped inside of a multiprocessing module the global variable won't be exported globally into the program but rather only inside the wrapped L2 method of this class
    global title
    global categoryResult
    print("[QA Automata Optimization Debug] : Called to analyze text")
    NLPProcessor.initMainEngine() #initialize gptM variable
    #fetch two
    print("[QA Automata Optimization Debug] : Creating Short Story")
    shortStory = NLPProcessor.generateShortStory(prompt)
    print("[QA Automata Optimization Debug] : Creating Title from Short Story")
    title = NLPProcessor.textFetchTitle(shortStory)
    print("[QA Automata Optimization Debug] : Categorizing from the Title of the story")
    categoryResult = NLPProcessor.textCategorizationAnalysis(title)

  def generateShortStory(prompt):
    filteredPrompt = prompt.replace("", ",")
    promptSubmission = "Create bedtime short story with these criteria" + " " + str(filteredPrompt) + "."
    promptResultRaw = gptM.prompt(str(promptSubmission))
    print("[QA Automata Optimization Debug] : Raw short story", promptResultRaw)
    return promptResultRaw

  def textFetchTitle(prompt):
    #promptSubmission = "Create art title from this prompt, Give short answer without any additional reasoning, do not change the context of the prompt, maximum 7 words." + " " + str(prompt)
    promptSubmission = "Create story article from this story:" + " " + str(prompt)
    promptResultRaw = gptM.prompt(str(promptSubmission))
    print("[QA Automata Optimization Debug] : Raw", promptResultRaw)
    filteredResult = promptResultRaw.replace(' ', '_') #Replaces Spaces with underscores
    filteredResult = re.sub('[^A-Za-z_]+', '', filteredResult) #filters out non alphabethical prompt result
    print("[QA Automata Optimization Debug] : Raw", filteredResult)
    return filteredResult

  def textCategorizationAnalysis(prompt):
    #Also Nah i didn't put full fat chatGPT like feature into the mix due to memory management? Right? right?
    #help
    #define 3 category such as (Realistic, Anime, Abstracts), with each category uses different type of model
    #if the model doesn't exists, Download the model
    #if not os.path.exists("gpt4all-lora-quantized.bin"):
        #print("[QA Automata Diffusion]: Downloading NLP Language Model")
        #wget.urlretrieve("https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized.bin", "gpt4all-lora-quantized.bin", MyProgressBar())
    # Actually this is unecessary due to the fact that the module will automatically download the default NLP model by itself

    promptSubmission = "If I were define 3 category (Realistic, Anime, Abstract) whats this prompt definitely falls into? Give one word answer without reason and only use the category that being given." + "\"" + str(prompt) + "\""
    promptResultRaw = gptM.prompt(str(promptSubmission))
    print("[QA Automata Optimization Debug] : Raw", promptResultRaw)
    categoryResult = ""
    #Filter out the result
    if "realis" in promptResultRaw:
      categoryResult = "Realistic"
    if "anime" in promptResultRaw:
      categoryResult = "Anime"
    if "abstract" in promptResultRaw:
      categoryResult = "Abstract"
    #fallback if there isn't any of those three
    if len(categoryResult) == 0:
      print("[QA Automata Diffusion]: Falling Back as Categorization Function fails to do its job")
      categoryResult = "Anime"
    print("[QA Automata Optimization Debug] : The prompt is", categoryResult)
    return categoryResult

def textPromptManipulation(prompt, globalMode):
  global currTime
  global paintingStyle
  global qualityEnforcement
  global disablePostProcessing
  global disableMultiPass
  global specialFX
  global neg_prompt
  global forceModeModel
  try:
    forceModeModel
  except NameError:
    forceModeModel=0
  currTime = datetime.now()
  #globalMode is a additional parameter if this function were called outside of the image generation function
  # what this does is basically it only manipulate or add additional prompt if this function was called through the main image generation functionor lets just say like inside generateImageFromText() rather than the processT2I wrapper sort of like that
  # this helps to reduce the required prompt token quota usage that need to be passed to the main function
  if globalMode == 0:
    qualityEnforcement = qualityEnforcement #beginning of the prompt injection and "," seperation handled by the middle part of the injection
    paintingStyle = ", " + paintingStyle + ", " #because this positioned in the middle its need to have , seperation both right and left
    specialFX = ", " + specialFX #end of the prompt but after the main user input prompt
    # append all prompt ( User and default quality injection )
    appendedPrompt=str(qualityEnforcement) + str(paintingStyle) + str(prompt) + str(specialFX)
  else:
    appendedPrompt=prompt
  # Model Selection and Ratio Enforcement (Can be overriden by the Global Ratio Overriding Variable or ratioOverride )
  if "forcePortraitMode" in prompt:
    forceModeModel=1 #forceModeModel 1 is reserved for Portrait mode Model
    print("ForceModeModel ", forceModeModel)
    appendedPrompt=appendedPrompt.replace('forcePortraitMode', '')
  if "forceLandscapeMode" in prompt:
    forceModeModel=2 #forceModeModel 2 is reserved for LandscapeMode
    appendedPrompt=appendedPrompt.replace('forceLandscapeMode', '')
    print("ForceModeModel ", forceModeModel)
  if "forceBalanceMode" in prompt:
    forceModeModel=3 #forceModeModel 3 is reserved for Balanced MidJourney Style Mode Model
    appendedPrompt=appendedPrompt.replace('forceBalanceMode', '')
    print("ForceModeModel ", forceModeModel)
  if "forceVectorMode" in prompt:
    forceModeModel=4 #forceModeModel 4 is reserved for Balanced MidJourney Vector Style Mode Model
    appendedPrompt=appendedPrompt.replace('forceVectorMode', '')
    print("ForceModeModel ", forceModeModel)
  if "forcePix2PixMode" in prompt:
    forceModeModel=5 #forceModeModel 5 is reserved for Pix2Pix Pipeline Mode Model
    appendedPrompt=appendedPrompt.replace('forcePix2PixMode', '')
    print("ForceModeModel ", forceModeModel)
  if "forceRealismMode" in prompt:
    forceModeModel=6 #forceModeModel 6 is reserved for Realism Pipeline Mode Model
    appendedPrompt=appendedPrompt.replace('forceRealismMode', '')
    print("ForceModeModel ", forceModeModel)
  if "forceLogoMode" in prompt:
    forceModeModel=7 #forceModeModel 7 is reserved for logo tuned Mode
    appendedPrompt=appendedPrompt.replace('forceRealismMode', '')
    print("ForceModeModel ", forceModeModel)

  if "disablequalityenforcement" in prompt:
    appendedPrompt=appendedPrompt.replace(qualityEnforcement, '')
    logging.debug("[QA Automata Optimization Debug] : Quality Enforcement Injection Disabled!, Generated Image will have worst quality")
    appendedPrompt=appendedPrompt.replace('disablequalityenforcement', '')
  if "disablesfx" in prompt:
    logging.debug("[QA Automata Optimization Debug] : Special Effects Enforcement Injection Disabled!, Generated Image will have generic quality")
    appendedPrompt=appendedPrompt.replace(specialFX, '')
    appendedPrompt=appendedPrompt.replace('disablesfx', '')
  if "disable_painting_style_enforcement" in prompt:
    logging.debug("[QA Automata Optimization Debug] : Painting style enforcement disabled, Generated Image will have inconsistent style")
    appendedPrompt=appendedPrompt.replace(paintingStyle, '')
    appendedPrompt=appendedPrompt.replace('disable_painting_style_enforcement', '')
  if "disable_multipass_prompt_mode" in prompt:
    logging.debug("[QA Automata Optimization Debug] : multipass mode disabled, your prompt will be limited to ~25 token")
    disableMultiPass=1
    appendedPrompt=appendedPrompt.replace('disable_multipass_prompt_mode', '')
  else:
    disableMultiPass=0
  if "disable_sharpness_cheat" in prompt:
    logging.debug("[QA Automata Optimization Debug] : Image will be outputted as the reality that the gpu can handle!")
    disablePostProcessing=1
    appendedPrompt=appendedPrompt.replace('disable_sharpness_cheat', '')
  else:
    disablePostProcessing=0
  neg_prompt_old=neg_prompt
  if "disable_quality_fence" in prompt:
    neg_prompt_old=neg_prompt
    neg_prompt=""
    logging.debug("[QA Automata Optimization Debug] : Minimum quality fence has been disabled, generated image will have unexpected or unlogical object very often!")
    appendedPrompt=appendedPrompt.replace('disable_quality_fence', '')
  else:
    neg_prompt=neg_prompt_old
  unwantedPromptNoise=["  ", ", ,", ",,", " , , "]
  for targetedNoise in unwantedPromptNoise:
    while targetedNoise in appendedPrompt:
      appendedPrompt=appendedPrompt.replace(targetedNoise, ",")
      #fix double "," or comas cleanup
  logging.debug("[QA Automata Optimization Debug] : Modified Prompt inside Function :", appendedPrompt)
  return appendedPrompt


# -------------------------------------------------------Dynamic Variable Functions
def reqOptimalResolutionAndModel(promptAnalyze, multicycleoptimization):
  global personPortraitKeyword
  global landscapeKeyword
  global optimalModel
  global megsPixelConstantDiffusion
  global parameterSafetyCheckerModifiable
  global lastAnalyzedWeight
  parameterSafetyCheckerModifiable = False
  if multicycleoptimization == 1:
    pass
    maxCap = trainedModelPxQuota
  else:
    maxCap = maximumPixelQuota
  pixelRenderingCap=freeMemoryCheck()/megsPixelConstantDiffusion #Maximum Pixel Rendering Quota
  if pixelRenderingCap > maxCap:
    logging.debug("[QA Automata Optimization Debug] : The quota is above the maximum Pixel quota that the model can handle!", pixelRenderingCap)
    pixelRenderingCap=maxCap
  logging.debug("[QA Automata Optimization Debug] : The Processor max Quota is ", pixelRenderingCap)
  #theres 3 modes or 3 aspect ratio
  # array [0] for xratio array [1] for yratio
  ratio=[width_Ratio, height_Ratio] #initialize array
  # Portrait optimized for Person Rendering 6:20
  # Balanced 18:7
  # Landscape Optimized 21:5
  # array [0] for xresolution array [1] for yresolution
  #Optimization Decision will be based on the weight of the detected keyword prompt
  #Viewfinder Resolution Render Limit Optimization
  #Since Gradio UI isn't a Apple product the developer isn't too focus on optimization but rather a sloppy handwork of UI/UX (literally from a efficient jpeg submitted into the UI convert back into png and locks up the fuck ?)
  #So what we going to do is limit Gradio viewing texture lowspec res by max width 720 pixel and Height 227 pixel
  #we decided to abandoned Gradio UI, its beyond broken
  #----------------------

  try:
    lastAnalyzedWeight
  except IndexError or NameError:
    raise ReAIError("ERROR! Did you forgot to call promptHandler function before calling reqOptimalResolutionAndModel?")
    exit()

  #check whether its the last pass usually passed with 0 prompts
  #[0] is for portrait and [1] is for landscape
  if promptAnalyze == "":
    try:
      optimalModel
    except NameError:
      print("optimalModel is not defined? That's weird")
      print("Falling back to fallback default model")

  portraitWeight = lastAnalyzedWeight[0]
  landscapeWeight = lastAnalyzedWeight[1]
  print("[QA Automata Optimization Debug] :", forceModeModel)
  if portraitWeight > landscapeWeight or forceModeModel == 1 :
    logging.debug("[QA Automata Optimization Debug] : Optimizing for Portrait")
    #4:11 starting Point
    ratio[0]=3+landscapeWeight
    ratio[1]=10+portraitWeight
    megsPixelConstantDiffusion=0.0086 #each model has its own constant to for its memory usage
    #optimalModel=""
    #License : creativeml-openrail-m
    #optimalModel="darasuke/animix_diffusers" #Based on animix https://huggingface.co/OedoSoldier/animix so the license will be based on the original source
    #optimalModel="Linaqruf/anything-v3.0"
    # This license is based on the civitAI and it is better than the fucking anything-v4
    optimalModel="stablediffusionapi/anything-v5" #https://civitai.com/models/9409?modelVersionId=33672
    parameterSafetyCheckerModifiable = True
  if landscapeWeight > portraitWeight or forceModeModel == 2 :
    logging.debug("[QA Automata Optimization Debug] : Optimizing for Landscape")
    #11:4 starting Point
    ratio[0]=8+landscapeWeight
    ratio[1]=4+portraitWeight
    megsPixelConstantDiffusion=0.0068 #each model has its own constant to for its memory usage
    #optimalModel="gsdf/Counterfeit-V2.5"
    #License : creativeml-openrail-m
    #optimalModel="uf/counterfeit-v3" #diffusers version of counterfeit-v3 (the original source of model : https://huggingface.co/gsdf/Counterfeit-V3.0  so the license will be based on the original source)
    optimalModel="gsdf/Counterfeit-V2.5"
    parameterSafetyCheckerModifiable = False
    #optimalModel="stablediffusionapi/counterfeit-v30" #its just a ported model of a non compatible counterfeit v3.0
  if landscapeWeight == portraitWeight or forceModeModel == 3 :
    logging.debug("[QA Automata Optimization Debug] : Optimizing for Balanced Applications")
    #9:7 starting point
    ratio[0]=9+landscapeWeight
    ratio[1]=7+portraitWeight
    #License : creativeml-openrail-m
    megsPixelConstantDiffusion=0.0068 #each model has its own constant to for its memory usage
    optimalModel="prompthero/openjourney"
    parameterSafetyCheckerModifiable = False

  #special case
  if forceModeModel == 4 :
    logging.debug("[QA Automata Optimization Debug] : Optimizing for Balanced Applications")
    #6:6 starting point
    ratio[0]=6+landscapeWeight
    ratio[1]=6+portraitWeight
    megsPixelConstantDiffusion=0.0269
    #License : creativeml-openrail-m
    optimalModel="stablediffusionapi/vector-art"
    parameterSafetyCheckerModifiable = False

  if forceModeModel == 5 :
    logging.debug("[QA Automata Optimization Debug] : Optimizing for Pix2Pix Enhancing Image Processor")
    print("This mode is forced to be developed due to microsoft chkdsk fixing the data by deleting the whole thing, Go fuck yourself Microsoft and Windows")
    #6:6 starting point
    ratio[0]=6+landscapeWeight
    ratio[1]=6+portraitWeight
    megsPixelConstantDiffusion=0.0869
    #License : creativeml-openrail-m
    optimalModel="radames/instruct-pix2pix-img2img"
    parameterSafetyCheckerModifiable = False

  if forceModeModel == 6 :
    logging.debug("[QA Automata Optimization Debug] : Optimizing for Realism Image Processor")
    print("This mode is forced to be developed due to microsoft chkdsk fixing the data by deleting the whole thing, Go fuck yourself Microsoft and Windows")
    #6:6 starting point
    ratio[0]=6+landscapeWeight
    ratio[1]=6+portraitWeight
    megsPixelConstantDiffusion=0.0068
    #License : creativeml-openrail-m
    optimalModel="SG161222/Realistic_Vision_V4.0"
    parameterSafetyCheckerModifiable = False

  if forceModeModel == 7 :
    logging.debug("[QA Automata Optimization Debug] : Optimizing for Logo Applications")
    #6:6 starting point
    ratio[0]=6+landscapeWeight
    ratio[1]=6+portraitWeight
    megsPixelConstantDiffusion=0.0269
    #License : creativeml-openrail-m
    optimalModel="nicky007/stable-diffusion-logo-fine-tuned"
    parameterSafetyCheckerModifiable = False

  try:
    ratioOverride
    ratio[0]=ratioOverride[0]
    ratio[1]=ratioOverride[1]
  except NameError:
    print("[QA Automata Optimization Debug] : No, override detected")
  except IndexError:
    print("[QA Automata Optimization Debug] : Confusing... but will continue")


  internalResolution=[1, 1, 0, 0, 1] # initialize array or list? [width, length, oldwidth, oldlength, shitGradioDownscaleMultiplier]
  #internalResolution[0]=round(pow(((pixelRenderingCap*ratio[0])/9), 1/2)) - (round(pow(((pixelRenderingCap*ratio[0])/9), 1/2)) % 8)
  #internalResolution[1]=round(pow((((pixelRenderingCap-internalResolution[0])*ratio[1])/9), 1/2)) - (round(pow((((pixelRenderingCap-internalResolution[0])*ratio[1])/9), 1/2)) % 8)
  internalResolution[0]=round(pow(((pixelRenderingCap*ratio[0])/9), 1/2) - (pow(((pixelRenderingCap*ratio[0])/9), 1/2)) % 8)
  internalResolution[1]=round(pow((((pixelRenderingCap-internalResolution[0])*ratio[1])/9), 1/2) - (pow((((pixelRenderingCap-internalResolution[0])*ratio[1])/9), 1/2)) % 8)
  if enable_experimental_quality_boost == 1 and enable_experimental_memory_management == 1:
    logging.debug("[QA Automata Optimization Debug] : experimental quality boost is currently enabled!")
    internalResolution[0]=internalResolution[0]*experimental_factorQualityBoost
    internalResolution[1]=internalResolution[1]*experimental_factorQualityBoost
  print("[QA Automata Optimization Debug] : The Processor Unit Only Capable Rendering at ", internalResolution[0], '*', internalResolution[1])
  shitGradioViewfinderRescaleDivision = 1
  widthShitGradioLimit=720
  heightShitGradioLimit=540
  if (internalResolution[0]) > widthShitGradioLimit:
    shitGradioViewfinderRescaleDivision = (internalResolution[0])/widthShitGradioLimit
  if (internalResolution[0]) > heightShitGradioLimit:
    shitGradioViewfinderRescaleDivision = (internalResolution[0])/widthShitGradioLimit
  internalResolution[4] = shitGradioViewfinderRescaleDivision
  logging.debug("[QA Automata Optimization Debug] : Optimizing Shit Gradio UI Performance Viewfinder Rescale Factor...", internalResolution[4])
  #upscaler Optimal Scaling Ratio
  return internalResolution




#---------------------------------------------- Main Image Generation Processing Function
imageLoadPhase1=0
imageLoadPhase2=0
def generateImageFromText(prompt, multicycleOptimization, guidance=guidance, steps=steps, img=None, strength=style_strength, auto_prefix=False):
    global imageGlobalBridge
    global seed

    logging.debug("[QA Automata Optimization Debug] : Using Discipline Guidance", guidance)
    appendedPrompt = textPromptManipulation(prompt, 0)
    logging.debug("[QA Automata Optimization Debug] : Modified Prompt main Function :", appendedPrompt)
    logging.debug("[QA Automata Optimization Debug] : Request Optimal Resolution...")
    optimalResolution=reqOptimalResolutionAndModel(prompt, multicycleOptimization) #requesting the optimal Resoluton array [0] xres [1] yres
    # Retrieve optimalModel Global Variable that was set from reqOptimalResolutionAndModel

    global selectedModel
    try:
      optimalModel
    except NameError:
      selectedModel = predefinedModelTarget
    else:
      selectedModel = optimalModel

    generateEngine_L1.initT2I_model()

    logging.debug("[QA Automata Optimization Debug] : Using Model", model_id)

    #replace this with seed = global variable that is handled with the L1 Generate function

    #replaced with global seed variable that is handled by the L1 Method
    """if randomize_seed == 1:
      seed = random.randint(2700000000000,2739896299199)
      logging.debug("[QA Automata Optimization Debug] : Randomized Seed ", seed)
    else:
      seed = globalStaticSeed"""


    lengthWord=len([word for word in appendedPrompt.split() if word.isalpha()])
    if dynamic_steps == 1:
      if device == "cuda":
        steps = lengthWord * 2 #https://stackoverflow.com/questions/19411767/split-python-word-count
        #steps = random.randint(27,69)
        if steps < 40:
          steps=40
        if steps > 69:
          steps=69
        logging.debug("[QA Automata Optimization Debug] : Optimizing Dynamic steps", steps)
      else:
         #steps = random.randint(6,27)
        steps = lengthWord * 2
        if steps < 13:
          steps=13
        if steps > 19:
          steps=19
        logging.debug("[QA Automata Optimization Debug] : Optimizing Dynamic steps", steps)
      if lengthWord > 25:
        logging.debug("[QA Automata Optimization Debug] : Quality Degradation may be occoured if you input too many prompt")
      if dynamic_guidance == 1:
        guidance=lengthWord
        if guidance > 14:
          guidance=13
        if guidance < 6:
          guidance=6
        logging.debug("[QA Automata Optimization Debug] : Optimizing Dynamic CFG", guidance)
    generator = torch.Generator(device).manual_seed(seed) if seed != 0 else None
    if device == "cuda":
      with autocast(device): #https://huggingface.co/stabilityai/stable-diffusion-2-1/discussions/9 Autocast only works for GPU NVIDIAAAH proprietary environment
        # create a randomly generated Seed (override the hyperparameter) Comment this part to determine your own seed from the hyper parameters. Same goes to the steps
        logging.debug("[QA Automata Optimization Debug] :", appendedPrompt)
        if enable_experimental_memory_management == 1:
          logging.debug("[QA Automata Optimization Debug] : CUDA Allocated Memory Management ", str(torch.cuda.memory_allocated()))
          logging.debug("[QA Automata Optimization Debug] : CUDA Reserve Memory Management ", str(torch.cuda.memory_reserved()))
        images_generated = pipe(
        appendedPrompt,
        negative_prompt = neg_prompt,
        num_inference_steps = steps,
        guidance_scale = guidance,
        width = optimalResolution[0], #xres
        height = optimalResolution[1], #yres
        generator = generator).images[0] #image generator
        #write_cache
        submittedImage = [] # a fix to subscriptable issue https://huggingface.co/spaces/stabilityai/stable-diffusion/resolve/9d961cf60d5bae28abbbe52dc003204a07181167/app.py
        #images_generated.save(f"/tmp/imagephase0.png") #wasteful of memory space
        #upscaler
        shared_memory['generatedDirectPreview'] = images_generated
        imageGlobalBridge = images_generated #fill the variable before decision
        if multicycleOptimization != 1:
          if disablePostProcessing == 0 :
            imageFlushMemoryStorageFlush()
            upscalerResolutionCheatingInit(optimalResolution[0], optimalResolution[1]) #initialize the upscaler Cheat
            images_generated.save("/tmp/before_processed.jpg")
            images_postProcessed = imagePostProcessing(images_generated) #outputs as a numpy opencv
            #Automatically Download to local comp
            fileName=versionProgram+"_"+str(seed)+"_"+str(currTime)
            saveFile=str("./"+ str(fileName) +".heic")
            print("[QA Automata Diffusion] : Saving File, (This might take a while) ", saveFile)
            register_heif_opener() #specifically to make PIL or pillow able to interact with HEIF or HEIC
            #cv2.imwrite(saveFile, images_postProcessed) # for saving files to the working directory
            writeImage(images_postProcessed, saveFile) #Accept only opencv2 numpy format then outputs it on an file, and it has no return value, so if you want to retrieve the image you have to read the image saveFile
            #writeExifImage(saveFile)
            print("[QA Automata Diffusion] : If your device mostly Android, can't open this picture try other device that is not Android")
            print("[QA Automata Diffusion] : We won't provide any compatibility to device that can't open the Picture, blame your vendor and developer to not support such open standard")
            #reduce Load Time by loading the jpg version
            images_postProcessed=image.open(saveFile).convert('RGB')
            imageGlobalBridge = images_postProcessed #saves the final result of the post processed iamge
            #load it again for viewing at the UI
            #images_postProcessed=image.open("/tmp/imagephase1.png").convert('RGB')
            imageRescaledForShitGradio = images_postProcessed.resize((round((optimalResolution[0])/optimalResolution[4]), round((optimalResolution[1])/optimalResolution[4])), image.ANTIALIAS)
            logging.debug("[QA Automata Optimization Debug] : Rescaling Image for viewfinder purposes", imageRescaledForShitGradio.size, " With Rescale", optimalResolution[4])
            submittedImage.append(imageRescaledForShitGradio) #images.append change to something that is low resolution and low specification for the viewfinder
          else:
            fileName=versionProgram+"_"+str(seed)+"_"+str(currTime)
            saveFile=str("./"+ str(fileName) +".heic")
            print("[QA Automata Diffusion] : Saving File, (This might take a while) ", saveFile)
            register_heif_opener() #specifically to make PIL or pillow able to interact with HEIF or HEIC
            images_postProcessed = images_generated
            imageGlobalBridge = images_postProcessed #saves the final result of the post processed iamge
            cv2.imwrite(saveFile, images_postProcessed) # for saving files to the working directory
            print("[QA Automata Diffusion] : If your device mostly Android, can't open this picture try other device that is not Android")
            print("[QA Automata Diffusion] : We won't provide any compatibility to device that can't open the Picture, blame your vendor and developer to not support such open standard")
            #reduce Load Time by loading the jpg version
            images_postProcessed=image.open(saveFile).convert('RGB')
            imageRescaledForShitGradio = images_postProcessed.resize((round((optimalResolution[0])/optimalResolution[4]), round((optimalResolution[1])/optimalResolution[4])), image.ANTIALIAS)
            logging.debug("[QA Automata Optimization Debug] : Disabling Sharpness Cheat!")
            logging.debug("[QA Automata Optimization Debug] : Rescaling Image for viewfinder purposes", imageRescaledForShitGradio.size, " With Rescale", optimalResolution[4])
            submittedImage.append(imageRescaledForShitGradio) #images.append change to something that is low resolution and low specification for the viewfinder

    else:
      # create a randomly generated Seed (override the hyperparameter) Comment this part to determine your own seed from the hyper parameters. Same goes to the steps
      logging.debug("[QA Automata Optimization Debug] :", appendedPrompt)
      images_generated = pipe(
      appendedPrompt,
      negative_prompt = neg_prompt,
      num_inference_steps = steps,
      guidance_scale = guidance,
      width = optimalResolution[0], #xres
      height = optimalResolution[1], #yres
      generator = generator).images[0] #image generator
      #write_cache
      submittedImage = []
      #images_generated.save(f"/tmp/imagephase0.png") #wasteful of memory space
      #upscaler
      shared_memory['generatedDirectPreview'] = images_generated
      imageGlobalBridge = images_generated
      if multicycleOptimization != 1:
        if disablePostProcessing == 0:
          imageFlushMemoryStorageFlush()
          upscalerResolutionCheatingInit(optimalResolution[0], optimalResolution[1]) #initialize the upscaler Cheat
          logging.debug("[QA Automata Optimization Debug] : Adding some finishing touches Please wait... ")
          images_generated.save("/tmp/before_processed.jpg")
          images_postProcessed = imagePostProcessing(images_generated) #openCV numpy format
          #Automatically Download to local comp
          fileName=versionProgram+"_"+str(seed)+"_"+str(currTime)
          saveFile=str("./"+ str(fileName) +".heic")
          logging.debug("[QA Automata Optimization Debug] : Saving File, (This might take a while)... ", saveFile)
          register_heif_opener() #specifically to make PIL or pillow able to interact with HEIF or HEIC
          #cv2.imwrite(saveFile, images_postProcessed) # for saving files to the working directory
          writeImage(images_postProcessed, saveFile) #Accept only opencv2 numpy format then outputs it on an file, and it has no return value, so if you want to retrieve the image you have to read the image saveFile
          ##writeExifImage(saveFile)
          print("[QA Automata Diffusion] : If your device mostly Android, can't open this picture try other device that is not Android")
          print("[QA Automata Diffusion] : We won't provide any compatibility to device that can't open the Picture, blame your vendor and developer to not support such open standard")
          images_postProcessed=image.open(saveFile).convert('RGB')
          imageGlobalBridge = images_postProcessed #saves the final result of the post processed iamge PIL format RGB
          #load it again for viewing at the UI
          #images_postProcessed=image.open("/tmp/imagephase1.png").convert('RGB')
          imageRescaledForShitGradio = images_postProcessed.resize((round((optimalResolution[0])/optimalResolution[4]), round((optimalResolution[1])/optimalResolution[4])), image.ANTIALIAS)
          logging.debug("[QA Automata Optimization Debug] : Rescaling Image for viewfinder purposes", imageRescaledForShitGradio.size, " With Rescale", optimalResolution[4])
          submittedImage.append(imageRescaledForShitGradio) #images.append change to something that is low resolution and low specification for the viewfinder
        else:
          fileName=versionProgram+"_"+str(seed)+"_"+str(currTime)
          saveFile=str("./"+ str(fileName) +".heic")
          print("[QA Automata Diffusion] : Saving File, (This might take a while) ", saveFile)
          register_heif_opener() #specifically to make PIL or pillow able to interact with HEIF or HEIC
          images_postProcessed = images_generated
          imageGlobalBridge = images_postProcessed #saves the final result of the post processed iamge
          cv2.imwrite(saveFile, images_postProcessed) # for saving files to the working directory
          print("[QA Automata Diffusion] : If your device mostly Android, can't open this picture try other device that is not Android")
          print("[QA Automata Diffusion] : We won't provide any compatibility to device that can't open the Picture, blame your vendor and developer to not support such open standard")
          images_postProcessed=image.open(saveFile).convert('RGB')
          imageRescaledForShitGradio = images_postProcessed.resize((round((optimalResolution[0])/optimalResolution[4]), round((optimalResolution[1])/optimalResolution[4])), image.ANTIALIAS)
          logging.debug("[QA Automata Optimization Debug] : Disabling Sharpness Cheat!")
          logging.debug("[QA Automata Optimization Debug] : Rescaling Image for viewfinder purposes", imageRescaledForShitGradio.size, " With Rescale", optimalResolution[4])
          submittedImage.append(imageRescaledForShitGradio) #images.append change to something that is low resolution and low specification for the viewfinder
    logging.debug("[QA Automata Optimization Debug] : Submitting Image for viewfinder purposes ")
    return submittedImage

def generateImageFromImage(prompt, multicycleOptimization, img, guidance=guidance, steps=steps, strength=style_strength_img2img, auto_prefix=False):
    global imageGlobalBridge
    global seed

    logging.debug("[QA Automata Optimization Debug] : Using Discipline Guidance", guidance)
    appendedPrompt = textPromptManipulation(prompt, 0)
    logging.debug("[QA Automata Optimization Debug] : Modified Prompt main Function :", appendedPrompt)
    logging.debug("[QA Automata Optimization Debug] : Request Optimal Resolution...")
    optimalResolution=reqOptimalResolutionAndModel(prompt, multicycleOptimization) #requesting the optimal Resoluton array [0] xres [1] yres
    # Retrieve optimalModel Global Variable that was set from reqOptimalResolutionAndModel

    global selectedModel
    try:
      optimalModel
    except NameError:
      selectedModel = predefinedModelTarget
    else:
      selectedModel = optimalModel

    generateEngine_L1.initI2I_model()

    logging.debug("[QA Automata Optimization Debug] : Using Model", model_id)
    #replaced with global seed variable that is handled by the L1 Method
    lengthWord=len([word for word in appendedPrompt.split() if word.isalpha()])
    if dynamic_steps == 1:
      if device == "cuda":
        steps = lengthWord * 2 #https://stackoverflow.com/questions/19411767/split-python-word-count
        #steps = random.randint(27,69)
        if steps < 40:
          steps=40
        if steps > 69:
          steps=69
        logging.debug("[QA Automata Optimization Debug] : Optimizing Dynamic steps", steps)
      else:
         #steps = random.randint(6,27)
        steps = lengthWord * 2
        if steps < 13:
          steps=13
        if steps > 20:
          steps=20
        logging.debug("[QA Automata Optimization Debug] : Optimizing Dynamic steps", steps)
      if lengthWord > 25:
        logging.debug("[QA Automata Optimization Debug] : Quality Degradation may be occoured if you input too many prompt")
      if dynamic_guidance == 1:
        guidance=lengthWord
        if guidance > 14:
          guidance=13
        if guidance < 6:
          guidance=6
        logging.debug("[QA Automata Optimization Debug] : Optimizing Dynamic CFG", guidance)
    generator = torch.Generator(device).manual_seed(seed) if seed != 0 else None
    if device == "cuda":
      with autocast(device): #https://huggingface.co/stabilityai/stable-diffusion-2-1/discussions/9 Autocast only works for GPU NVIDIAAAH proprietary environment
        # create a randomly generated Seed (override the hyperparameter) Comment this part to determine your own seed from the hyper parameters. Same goes to the steps
        logging.debug("[QA Automata Optimization Debug] :", appendedPrompt)
        if enable_experimental_memory_management == 1:
          logging.debug("[QA Automata Optimization Debug] : CUDA Allocated Memory Management ", str(torch.cuda.memory_allocated()))
          logging.debug("[QA Automata Optimization Debug] : CUDA Reserve Memory Management ", str(torch.cuda.memory_reserved()))
        if multicycleOptimization != 1:
          # since we cant define our own resolution on the i2i pipe we have to resize it beforehand
          img = img.resize((optimalResolution[0], optimalResolution[1]), image.LANCZOS)
        images_generated = pipe_i2i(
        appendedPrompt,
        negative_prompt = neg_prompt,
        num_inference_steps = steps,
        guidance_scale = guidance,
        image = img,
        #width = optimalResolution[0], #xres
        #height = optimalResolution[1], #yres
        generator = generator).images[0] #image generator
        #write_cache
        submittedImage = [] # a fix to subscriptable issue https://huggingface.co/spaces/stabilityai/stable-diffusion/resolve/9d961cf60d5bae28abbbe52dc003204a07181167/app.py
        #images_generated.save(f"/tmp/imawgephase0.png") #wasteful of memory space
        #upscaler
        shared_memory['generatedDirectPreview'] = images_generated
        imageGlobalBridge = images_generated #fill the variable before decision (as the bridge for the multicycle processing)
        if multicycleOptimization != 1:
          if disablePostProcessing == 0 :
            imageFlushMemoryStorageFlush()
            upscalerResolutionCheatingInit(optimalResolution[0], optimalResolution[1]) #initialize the upscaler Cheat
            logging.debug("[QA Automata Optimization Debug] : Adding some finishing touches Please wait... ")
            images_generated.save("/tmp/before_processed.jpg")
            images_postProcessed = imagePostProcessing(images_generated)
            imageGlobalBridge = images_postProcessed #saves the final result of the post processed iamge
            #Automatically Download to local comp
            fileName=versionProgram+"_"+str(seed)+"_"+str(currTime)
            saveFile=str("./"+ str(fileName) +".heic")
            print("[QA Automata Diffusion] : Saving File, (This might take a while) ", saveFile)
            register_heif_opener() #specifically to make PIL or pillow able to interact with HEIF or HEIC
            #cv2.imwrite(saveFile, images_postProcessed) # for saving files to the working directory
            writeImage(images_postProcessed, saveFile) #Accept only opencv2 numpy format then outputs it on an file, and it has no return value, so if you want to retrieve the image you have to read the image saveFile
            ##writeExifImage(saveFile)
            print("[QA Automata Diffusion] : If your device mostly Android, can't open this picture try other device that is not Android")
            print("[QA Automata Diffusion] : We won't provide any compatibility to device that can't open the Picture, blame your vendor and developer to not support such open standard")
            #reduce Load Time by loading the jpg version
            images_postProcessed=image.open(saveFile).convert('RGB')
            imageGlobalBridge = images_postProcessed #saves the final result of the post processed iamge
            #load it again for viewing at the UI
            #images_postProcessed=image.open("/tmp/imagepDhase1.png").convert('RGB')
            imageRescaledForShitGradio = images_postProcessed.resize((round((optimalResolution[0])/optimalResolution[4]), round((optimalResolution[1])/optimalResolution[4])), image.ANTIALIAS)
            logging.debug("[QA Automata Optimization Debug] : Rescaling Image for viewfinder purposes", imageRescaledForShitGradio.size, " With Rescale", optimalResolution[4])
            submittedImage.append(imageRescaledForShitGradio) #images.append change to something that is low resolution and low specification for the viewfinder
          else:

            fileName=versionProgram+"_"+str(seed)+"_"+str(currTime)
            saveFile=str("./"+ str(fileName) +".heic")
            print("[QA Automata Diffusion] : Saving File, (This might take a while) ", saveFile)
            register_heif_opener() #specifically to make PIL or pillow able to interact with HEIF or HEIC
            images_postProcessed = images_generated
            cv2.imwrite(saveFile, images_postProcessed) # for saving files to the working directory
            print("[QA Automata Diffusion] : If your device mostly Android, can't open this picture try other device that is not Android")
            print("[QA Automata Diffusion] : We won't provide any compatibility to device that can't open the Picture, blame your vendor and developer to not support such open standard")
            #reduce Load Time by loading the jpg version
            images_postProcessed=image.open(saveFile).convert('RGB')
            imageRescaledForShitGradio = images_postProcessed.resize((round((optimalResolution[0])/optimalResolution[4]), round((optimalResolution[1])/optimalResolution[4])), image.ANTIALIAS)
            logging.debug("[QA Automata Optimization Debug] : Disabling Sharpness Cheat!")
            logging.debug("[QA Automata Optimization Debug] : Rescaling Image for viewfinder purposes", imageRescaledForShitGradio.size, " With Rescale", optimalResolution[4])
            submittedImage.append(imageRescaledForShitGradio) #images.append change to something that is low resolution and low specification for the viewfinder

    else:
      # create a randomly generated Seed (override the hyperparameter) Comment this part to determine your own seed from the hyper parameters. Same goes to the steps
      if multicycleOptimization != 1:
          # since we cant define our own resolution on the i2i pipe we have to resize it beforehand
          img = img.resize((optimalResolution[0], optimalResolution[1]), image.ANTIALIAS)
      logging.debug("[QA Automata Optimization Debug] :", appendedPrompt)
      images_generated = pipe_i2i(
      appendedPrompt,
      negative_prompt = neg_prompt,
      num_inference_steps = steps,
      guidance_scale = guidance,
      image = img,
      #width = optimalResolution[0], #xres
      #height = optimalResolution[1], #yres
      generator = generator).images[0] #image generator
      #write_cache
      submittedImage = []
      #images_generated.save(f"/tmp/imagephase0.png") #wasteful of memory space
      #upscaler
      shared_memory['generatedDirectPreview'] = images_generated
      imageGlobalBridge = images_generated#fill the variable before decision
      if multicycleOptimization != 1:
        if disablePostProcessing == 0 :
          imageFlushMemoryStorageFlush()
          upscalerResolutionCheatingInit(optimalResolution[0], optimalResolution[1]) #initialize the upscaler Cheat
          logging.debug("[QA Automata Optimization Debug] : Adding some finishing touches Please wait... ")
          images_generated.save("/tmp/before_processed.jpg")
          images_postProcessed = imagePostProcessing(images_generated) #outputs as cv2 numpy format
          #Automatically Download to local comp
          fileName=versionProgram+"_"+str(seed)+"_"+str(currTime)
          saveFile=str("./"+ str(fileName) +".hdr")
          print("[QA Automata Diffusion] : Saving File, (This might take a while) ", saveFile)
          #cv2.imwrite(saveFile, images_postProcessed) # for saving files to the working directory
          writeImage(images_postProcessed, saveFile) #Accept only opencv2 numpy format then outputs it on an file, and it has no return value, so if you want to retrieve the image you have to read the image saveFile
          #writeExifImage(saveFile)
          print("[QA Automata Diffusion] : If your device mostly Android, can't open this picture try other device that is not Android")
          print("[QA Automata Diffusion] : We won't provide any compatibility to device that can't open the Picture, blame your vendor and developer to not support such open standard")
          #reduce Load Time by loading the jpg version
          images_postProcessed=image.open(saveFile).convert('RGB')
          imageGlobalBridge = images_postProcessed #saves the final result of the post processed iamge
          #load it again for viewing at the UI
          #images_postProcessed=image.open("/tmp/imagephase1.png").convert('RGB')
          imageRescaledForShitGradio = images_postProcessed.resize((round((optimalResolution[0])/optimalResolution[4]), round((optimalResolution[1])/optimalResolution[4])), image.ANTIALIAS)
          logging.debug("[QA Automata Optimization Debug] : Rescaling Image for viewfinder purposes", imageRescaledForShitGradio.size, " With Rescale", optimalResolution[4])
          submittedImage.append(imageRescaledForShitGradio) #images.append change to something that is low resolution and low specification for the viewfinder
        else:
          fileName=versionProgram+"_"+str(seed)+"_"+str(currTime)
          saveFile=str("./"+ str(fileName) +".hdr")
          print("[QA Automata Diffusion] : Saving File, (This might take a while) ", saveFile)
          register_heif_opener() #specifically to make PIL or pillow able to interact with HEIF or HEIC
          images_postProcessed = images_generated
          cv2.imwrite(saveFile, images_postProcessed) # for saving files to the working directory
          print("[QA Automata Diffusion] : If your device mostly Android, can't open this picture try other device that is not Android")
          print("[QA Automata Diffusion] : We won't provide any compatibility to device that can't open the Picture, blame your vendor and developer to not support such open standard")
          #reduce Load Time by loading the jpg version
          images_postProcessed=image.open(saveFile).convert('RGB')
          imageRescaledForShitGradio = images_postProcessed.resize((round((optimalResolution[0])/optimalResolution[4]), round((optimalResolution[1])/optimalResolution[4])), image.ANTIALIAS)
          logging.debug("[QA Automata Optimization Debug] : Disabling Sharpness Cheat!")
          logging.debug("[QA Automata Optimization Debug] : Rescaling Image for viewfinder purposes", imageRescaledForShitGradio.size, " With Rescale", optimalResolution[4])
          submittedImage.append(imageRescaledForShitGradio) #images.append change to something that is low resolution and low specification for the viewfinder
    logging.debug("[QA Automata Optimization Debug] : Submitting Image for viewfinder purposes ")
    return submittedImage

def promptHandler(words):
  global artTitle
  global lastAnalyzedWeight
  #promptCategory, artTitle = NLPProcessor.AnalyzeText_L2(words)
  #test the NLPProcessor
  #Still horrible
  #NLPProcessor.AnalyzeText_L2(words)
  try:
    lastAnalyzedWeight
  except IndexError:
    lastAnalyzedWeight = [] #Initialize lastAnalyzedWeight Variable if its not initialized yet
  except NameError:
    lastAnalyzedWeight = [] #Initialize lastAnalyzedWeight Variable if its not initialized yet
  # Move this into the main prompt analyzer which analyze how many token there is
  #Analyze on what
  personPortraitKeyword=["girl", "boy", "shounen", "girl", "boy", "man", "women", "portrait", "waifu", "husband"]
  portraitWeight=0
  landscapeKeyword=["landscape", "garden", "shop", "city", "cityscape", "field", "environment", "space", "night", "sunlight", "star", "indoor", "outdoor", "room", "school", "office", "university"]
  landscapeWeight=0
  for x in personPortraitKeyword:
    if x in words:
      portraitWeight=portraitWeight+1
  for x in landscapeKeyword:
    if x in words:
      landscapeWeight=landscapeWeight+1
  #[0] for portrait weight whilst [1] for landscape weight keywords
  #save the analyzed prompt weight so later on the img can be consistent
  lastAnalyzedWeight.insert(0, portraitWeight)
  lastAnalyzedWeight.insert(1, landscapeWeight)
  #lastAnalyzedWeight[0] = portraitWeight
  #lastAnalyzedWeight[1] = landscapeWeight
  #this analyzed keyword weight will be globally by the reqOptimizedResolution and etc

  tokenSegment=words.split(",")
  tokenSegmentFocusCount=0
  tokenSegmentFocusCountPos=0
  sumRawToken=0 # the token that might be used in total
  submissionToken = ""
  combinedSubmissionToken=[]
  totalTokenSegment=0
  #count total segment
  for count in tokenSegment:
    totalTokenSegment = totalTokenSegment+1 #count total segment
  #appendedPrompt.replace(specialFX, '')
  for tokenSegmentFocus in tokenSegment:
    tokenSegmentFocusCount = len([word for word in tokenSegmentFocus.split() if word.isalpha()]) #count for the token that are inside of the segment
    #print(tokenSegmentFocusCount)
    sumRawToken = sumRawToken + tokenSegmentFocusCount
    if tokenSegmentFocusCount >= maxToken:
      pass
      logging.debug("[QA Automata Optimization Debug] : Segment [", tokenSegmentFocusCountPos, "] Overuse Quota!")
      #replace phrase space to _
      tokenSegmentFocusOld = tokenSegmentFocus
      tokenSegmentFocus = tokenSegmentFocus.replace(" ", "_")
      tokenSegmentFocus = tokenSegmentFocus.replace(tokenSegmentFocusOld, "")
    else:
      pass
      logging.debug("[QA Automata Optimization Debug] : Segment [", tokenSegmentFocusCountPos, "] Below the limit")
    # take art title from first tokenSegmentFocusCountPosition
    if tokenSegmentFocusCountPos == 0:
      artTitle = tokenSegmentFocus
    tokenSegmentFocusCountPos = tokenSegmentFocusCountPos + 1
    #print(tokenSegmentFocus)
    if submissionToken == "":
      submissionToken = submissionToken + tokenSegmentFocus
    else:
      submissionToken = submissionToken + "," + tokenSegmentFocus

    if (tokenSegmentFocusCountPos % maxTokenSegmentPerSubmission) == 0 and tokenSegmentFocusCountPos > 0:
      logging.debug("[QA Automata Optimization Debug] : Overuse Quota, Segmenting!====")
      #print(submissionToken) #this is the segmented token where it is being submitted to the pipeline
      combinedSubmissionToken.append(submissionToken)
      submissionToken = ""
    if (tokenSegmentFocusCountPos == totalTokenSegment): #check if we reached the end of the tokenSegment then don't forget to append it to combinedSubmissionToken
      combinedSubmissionToken.append(submissionToken)
  return combinedSubmissionToken

#post Process Image or Image Post Processing
def imagePostProcessing(image):
  logging.debug("[QA Automata Optimization Debug] : Upscaling Image")


  #before being Upscaled it need to be Adjusted or color Graded
  # https://pythonexamples.org/python-pillow-adjust-image-contrast/ base reference
  # Tool ImagePostProcessWorker
  #phase one : Contrast
  contrastWorker = ImagePostProcessWorker.Contrast(image)
  tuningConst = 0.6944
  image = contrastWorker.enhance(tuningConst)

  #Phase two : brightness
  brightnessWorker = ImagePostProcessWorker.Brightness(image)
  tuningConst = 1.0454
  image = brightnessWorker.enhance(tuningConst)

  #phase three : Sharpness

  sharpnessWorker = ImagePostProcessWorker.Sharpness(image)
  tuningConst = 1.16
  image = sharpnessWorker.enhance(tuningConst)

  #Phase Four : Saturation

  saturationWorker = ImagePostProcessWorker.Color(image)
  tuningConst = 1.194124
  image = saturationWorker.enhance(tuningConst)

  # TODO : Before Upscaling do gfpgan face restoration
  # https://github.com/TencentARC/GFPGAN


  # Upscale
  #img = image.open("/tmp/process1.png").convert('RGB') #FAIL
  #https://stackoverflow.com/questions/43232813/convert-opencv-image-format-to-pil-image-format
  #loadedThroughOpenCV2 = cv2.imread("/tmp/process1.png")
  #process2PILConv = cv2.cvtColor(loadedThroughOpenCV2, cv2.COLOR_BGR2RGB) #FAIL
  #process2PILConv = image.open("/tmp/process1.jpg").convert('RGB') #degraded mode but its probably still fail #yep it is
  #img = image.fromarray(process2PILConv)
  result = image
  result.save("/tmp/process1.png")
  shapeAnalysis = cv2.imread("/tmp/process1.png")


  #----------------------------------------------------------------------------------------- WATERMARK --------------------------------------------------------------------------------------

  # Add Imprint for Questandachievement Text on the image
  # Imprint Format (4 object)
  # Title (Taken from the first phrase of the Prompt) [Riesling]
  # Barcode (Barcode of the Seed) [Free 3 of 9]
  # Seed in number [F1.8]
  # ("Questandachievement Automata Industry") [F1.8]
  # ("dE_QARE 0.3 Beta") [F1.8]


  # Note : Input Image will be PIL Object Format result Variable
  requestSprites()
  requestFont() # request font extraction
  # Resolution Survey
  # imgh Height, imgw Width, imgc Color Channel
  # https://note.nkmk.me/en/python-opencv-pillow-image-size/

  # as designed on the GIMP software the texts are going to be on the left bottom side
  # we designed on a 8704x7680 canvas and because on how we designed the logo being imprinted on the bottom left with some margen
  # we focus on the 8704 width and uses 7680 as for the scaling of the font
  # Title Configuration ======
  # Positioning
  # at the canvas 8704 width the starting of the Title is on 603px ( we need to convert it to percentage to be adaptable on various of resolution ) (603/8704)*100% = 6.92784926470588235294%
  # at the canvas of 7680 height the position is 6210px (6210/7680) * 100% = 80.859375%
  # Font Size
  # as for the font size
  # as for the height of the Title since the height is 7680 but the font size of Title is 1024px 1:1 ratio (1024/7680) * 100% = 13.3 %
  # Font Riesling
  # Barcode Configuration =====
  # Positioning
  # at the canvas of 8704 width the starting of the Barcode the starting position should be aligned with Title which is 603px which also means position 6.92784926470588235294%
  # at the canvas of 7680 height the position is at 6648 (6648/7680)*100% = 86.5625%
  # font size
  # as for the font size we uses the height 7680 and the font size 384px (384/7680)*100 = 5%
  # Font Free3of9
  # Seed Number Configuration =====
  # at the canvas of 8704 starting position should be aligned with Title which is 603px which also means position 6.92784926470588235294%
  # at the canvas with 7680 height the mosition is at 6946 (6946/7680)*100 = 90%
  # font size
  # the font size is 210px 1:1 at using height 7680px as its point of measure (210/7680) * 100 = 2.734%
  # Font F1.8
  # WatermarkI =====
  # at the canvas of 8704 starting position should be aligned with Title which is 603px which also means position 6.92784926470588235294%
  # at the canvas with 7680 height the mosition is at 7188 (7188/7680)*100 = 93.59375%
  # font size
  # the font size is 128 1:1 at using height 7680px as its point of measure (128/7680) * 100 = 1.66666666666666666667%
  # Font F1.8
  # Watermark II =====
  # at the canvas of 8704 starting position should be aligned with Title which is 603px which also means position 6.92784926470588235294%
  # at the canvas with 7680 height the mosition is at 7329 (7329/7680)*100 = 95.4296875%
  # font size
  # the font size is 128 1:1 at using height 7680px as its point of measure (128/7680) * 100 = 1.66666666666666666667%
  # Font F1.8

  #Default Text
  TitleImprinted = artTitle
  SeedImprinted = str(seed)
  BarcodeImprinted = str(SeedImprinted)
  WatermarkIImprinted = "Questandachievement Automata Industry"
  WatermarkIIImprinted = versionProgram
  #https://towardsdatascience.com/adding-text-on-image-using-python-2f5bf61bf448
  # image object input "result" output object PIL "result"
  # Font Files (f18.otf, free3of9.ttf, riesling.ttf) #opentype

  #example
  #TitleImprinted_Font = ImageFont.truetype('./riesling.ttf', 200) (200 is the font size)
  imgh, imgw, imgc = shapeAnalysis.shape
  if watermarkStyle == 0:
    pass

  if watermarkStyle == 1: #default or Original
    TitleImprinted_Font = ImageFont.truetype('./riesling.ttf', (round(imgh*(10.4/100))))
    SeedImprinted_Font = ImageFont.truetype('./f18.otf', (round(imgh*(2.734/100))))
    BarcodeImprinted_Font = ImageFont.truetype('./free3of9.ttf', (round(imgh*(5/100))))
    WatermarkIImprinted_Font = ImageFont.truetype('./f18.otf', (round(imgh*(1.667/100))))
    WatermarkIIImprinted_Font = ImageFont.truetype('./f18.otf', (round(imgh*(1.667/100))))
    resultEditable = ImageDraw.Draw(result)
    #Render Example image_editable.text((15,15), title_text, (237, 230, 211), font=title_font) (15,15) (x,y) are the cartecian coord on the canvas while the 237, 230, 211 is the RGB color for the text
    #Title
    resultEditable.text((round(imgw*(6.927849/100)),round(imgh*(73.225937/100))), TitleImprinted, (0, 0, 0), font=TitleImprinted_Font)
    #Barcode
    resultEditable.text((round(imgw*(6.927849/100)),round(imgh*(85.5625/100))), BarcodeImprinted, (0, 0, 0), font=BarcodeImprinted_Font)
    #SeedImprinted
    resultEditable.text((round(imgw*(6.927849/100)),round(imgh*(90/100))), SeedImprinted, (0, 0, 0), font=SeedImprinted_Font)
    #WatermarkI
    resultEditable.text((round(imgw*(6.927849/100)),round(imgh*(93.59375/100))), WatermarkIImprinted, (0, 0, 0), font=WatermarkIImprinted_Font)
    #WatermarkII
    resultEditable.text((round(imgw*(6.927849/100)),round(imgh*(95.42968/100))), WatermarkIIImprinted, (0, 0, 0), WatermarkIIImprinted_Font)
    # this should be enough to render text onto the image

  if watermarkStyle == 2: #Barcode left and questandachievement (This might be a good preset for Printing the image)
  #Barcode and Questandachievement text bottom
    WatermarkIImprinted = "Q7"
    #TitleImprinted_Font = ImageFont.truetype('./riesling.ttf', (round(imgh*(10.4/100))))
    #SeedImprinted_Font = ImageFont.truetype('./f18.otf', (round(imgh*(2.734/100))))
    BarcodeImprinted_Font = ImageFont.truetype('./free3of9.ttf', (round(imgh*(3.27/100))))
    WatermarkIImprinted_Font = ImageFont.truetype('./f18.otf', (round(imgh*(1.22/100))))
    #WatermarkIIImprinted_Font = ImageFont.truetype('./f18.otf', (round(imgh*(1.667/100))))
    resultEditable = ImageDraw.Draw(result)
    #Render Example image_editable.text((15,15), title_text, (237, 230, 211), font=title_font) (15,15) (x,y) are the cartecian coord on the canvas while the 237, 230, 211 is the RGB color for the text
    #Title
    #resultEditable.text((round(imgw*(6.927849/100)),round(imgh*(73.225937/100))), TitleImprinted, (0, 0, 0), font=TitleImprinted_Font)
    #Barcode
    resultEditable.text((round(imgw*(6.927849/100)),round(imgh*(93.59375/100))), BarcodeImprinted, (0, 0, 0), font=BarcodeImprinted_Font)
    #SeedImprinted
    #resultEditable.text((round(imgw*(6.927849/100)),round(imgh*(90/100))), BarcodeImprinted, (0, 0, 0), font=SeedImprinted_Font)
    #WatermarkI
    resultEditable.text((round(imgw*(89/100)),round(imgh*(93.89375/100))), WatermarkIImprinted, (0, 0, 0), font=WatermarkIImprinted_Font)
    #WatermarkII
    #resultEditable.text((round(imgw*(6.927849/100)),round(imgh*(95.42968/100))), WatermarkIIImprinted, (0, 0, 0), WatermarkIIImprinted_Font)
    # this should be enough to render text onto the image

  if watermarkStyle == 3: #Barcode left and questandachievement (This might be a good preset for Printing the image)
  #Barcode and Questandachievement text bottom
    WatermarkPNGImprinted = image2.open("./q7Logo.png").convert('RGBA')
    BarcodeImprinted_Font = ImageFont.truetype('./free3of9.ttf', (round(imgh*(3.27/100))))
    WatermarkIImprinted_Font = ImageFont.truetype('./f18.otf', (round(imgh*(1.22/100))))
    #WatermarkIIImprinted_Font = ImageFont.truetype('./f18.otf', (round(imgh*(1.667/100))))
    resultEditable = ImageDraw.Draw(result)
    resultEditable.text((round(imgw*(8.327849/100)),round(imgh*(93.59375/100))), BarcodeImprinted, (0, 0, 0), font=BarcodeImprinted_Font)
    #resultEditable.text((round(imgw*(89/100)),round(imgh*(93.89375/100))), WatermarkIImprinted, (0, 0, 0), font=WatermarkIImprinted_Font)
    #replaced with Image Overlay
    generated_width, generated_height = result.size
    logo_width, logo_height = WatermarkPNGImprinted.size
    #scaling algorithm
    # Select the biggest px resolution from the logo height or the width for the comparison or factorLogoScale
    if logo_width > logo_height:
      logo_px_comparison = logo_width
    else:
      logo_px_comparison = logo_height
    # Select the generated_height as generated_px_comparison
    generated_px_comparison = generated_height
    factorLogoScale = round((generated_px_comparison/logo_px_comparison)*12)
    WatermarkPNGImprinted = WatermarkPNGImprinted.resize(((round(logo_width/factorLogoScale)),(round(logo_height/factorLogoScale))))
    mergedImage = image2.new('RGB', (generated_width, generated_height), (0, 0, 0, 0)) #doing it on full size cause the environment to starve memory
    mergedImage.paste(result, (0, 0))
    mergedImage.paste(WatermarkPNGImprinted, (round(imgw*(83/100)),round(imgh*(4.195/100))), mask=WatermarkPNGImprinted) #(posX, posY) for the size it need to be resized beforehand
    result = mergedImage

  #----------------------------------------------------------------------------------------- WATERMARK --------------------------------------------------------------------------------------

  # Note : Output to "image" variable and PIL object format
  result.save("/tmp/process2.png")
  result.save("/tmp/process2.jpg")
  #split gamma HI, MED, LO to create High Dynamic Range
  imgGammaSplit = cv2.imread("/tmp/process2.png")
  #https://stackoverflow.com/questions/57309612/how-to-change-the-exposure-of-the-photo-in-opencv
  #HI gamma table split img
  gamma=0.4
  gamma_table=[np.power(x/255.0,gamma)*255.0 for x in range(256)]
  gamma_table=np.round(np.array(gamma_table)).astype(np.uint8)
  imgGammaHI = cv2.LUT(imgGammaSplit,gamma_table)
  cv2.imwrite("/tmp/Preprocess1_gamma_HI.jpg", imgGammaHI)

  #MED gamma table split img
  gamma=1.0
  gamma_table=[np.power(x/255.0,gamma)*255.0 for x in range(256)]
  gamma_table=np.round(np.array(gamma_table)).astype(np.uint8)
  imgGammaMED = cv2.LUT(imgGammaSplit,gamma_table)
  cv2.imwrite("/tmp/Preprocess1_gamma_MED.jpg", imgGammaMED)

  #LO gamma table split img
  gamma=2.2
  gamma_table=[np.power(x/255.0,gamma)*255.0 for x in range(256)]
  gamma_table=np.round(np.array(gamma_table)).astype(np.uint8)
  imgGammaLO = cv2.LUT(imgGammaSplit,gamma_table)
  cv2.imwrite("/tmp/Preprocess1_gamma_LO.jpg", imgGammaLO)

  #----- Convert format for manipulation
  # openCV2 Format
  #https://docs.opencv.org/3.4/d2/df0/tutorial_py_hdr.html

  #print("[QA Automata Optimization Debug] : Converting Image into True HDR using OpenCV2")
  img_fn = ["/tmp/Preprocess1_gamma_LO.jpg", "/tmp/Preprocess1_gamma_MED.jpg", "/tmp/Preprocess1_gamma_HI.jpg"]
  img_list = [cv2.imread(fn) for fn in img_fn]
  exposure_times = np.array([0.0083, 0.025, 1.0], dtype=np.float32)


  logging.debug("[QA Automata Optimization Debug] : Converting Image into True HDR mertens Algorithm")
  tonemap1 = cv2.createTonemap(gamma=1.2)
  merge_mertens = cv2.createMergeMertens()
  res_mertens = merge_mertens.process(img_list)
  res_mertens_8bit = np.clip(res_mertens*255, 0, 255).astype('uint8')
  cv2.imwrite("/tmp/process3.hdr", res_mertens_8bit)
  cv2.imwrite("/tmp/process3.png", res_mertens_8bit)
  cv2.imwrite("/tmp/process3.jpg", res_mertens_8bit)
  HDRMerged = res_mertens

  #process4 Sharpening by using opencv2 CLAHE method
  #https://www.geeksforgeeks.org/automatic-color-correction-with-opencv-and-python/
  preprocessedImage2Sharpening = cv2.imread("/tmp/process3.png")
  sharpening = cv2.cvtColor(preprocessedImage2Sharpening, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(sharpening)
  # Apply CLAHE to the L channel
  SHARPENING_PHASE2_CLIPL=2.327
  clahe = cv2.createCLAHE(clipLimit=SHARPENING_PHASE2_CLIPL, tileGridSize=(2,2))
  l = clahe.apply(l)
  sharpening = cv2.merge((l, a, b))
  process4Done = cv2.cvtColor(sharpening, cv2.COLOR_LAB2BGR)
  cv2.imwrite("/tmp/process4.png", process4Done)

  #process5 Auto Brightness and Contrast Enhancement
  #https://stackoverflow.com/questions/56905592/automatic-contrast-and-brightness-adjustment-of-a-color-photo-of-a-sheet-of-pape

  gray = cv2.cvtColor(process4Done, cv2.COLOR_BGR2GRAY)

  # Calculate grayscale histogram
  hist = cv2.calcHist([gray],[0],None,[256],[0,256])
  hist_size = len(hist)

  # Calculate cumulative distribution from the histogram
  accumulator = []
  accumulator.append(float(hist[0]))
  for index in range(1, hist_size):
      accumulator.append(accumulator[index -1] + float(hist[index]))

  # Locate points to clip
  maximum = accumulator[-1]
  AUTO_CONTRAST = 0.42
  clip_hist_percent = AUTO_CONTRAST
  clip_hist_percent *= (maximum/100.0)
  clip_hist_percent /= 2

  # Locate left cut
  minimum_gray = 0
  while accumulator[minimum_gray] < clip_hist_percent:
      minimum_gray += 1

  # Locate right cut
  maximum_gray = hist_size -1
  while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
      maximum_gray -= 1

  # Calculate alpha and beta values
  alpha = 255 / (maximum_gray - minimum_gray)
  beta = -minimum_gray * alpha

  '''
  # Calculate new histogram with desired range and show histogram
  new_hist = cv2.calcHist([gray],[0],None,[256],[minimum_gray,maximum_gray])
  plt.plot(hist)
  plt.plot(new_hist)
  plt.xlim([0,256])
  plt.show()
  '''

  auto_result_Process5 = cv2.convertScaleAbs(process4Done, alpha=alpha, beta=beta)

  cv2.imwrite("/tmp/process5.png", auto_result_Process5)

  #Done upscale to ~10000px
  #TODO : Convert Image to 12-bit Image, Convert Image to HDR, add EXIF into the image
  #print("[QA Automata Optimization Debug] : reset image into openCV format")
  #result = cv2.imread("/tmp/process3.png")
  #upscale postprocessed Result
  result = image2.open("/tmp/process5.png")
  result = modelUpscaler.predict(result)
  result.save("/tmp/process6.png")

  #Final Result
  result = cv2.imread("/tmp/process6.png")
  cv2.imwrite("/tmp/processed.hdr", result) #full quality
  cv2.imwrite("/tmp/processed.jpg", result) #degraded_quality

  return result

def conv2Dto3D(image2D):
  logging.debug("[QA Automata Optimization Debug] : Stub implementation")
  #implement zero-1-to-3
  pass

def writeExifImage(file):
  # add EXIF Data https://towardsdatascience.com/read-and-edit-image-metadata-with-python-f635398cd991
  # call imgexif
  logging.debug("[QA Automata Optimization Debug] : Failure is imminent!")
  logging.debug("[QA Automata Optimization Debug] : adding Exif Data into the image")
  img_path = file

  with open(img_path, 'rb') as img_file:
    img = imgexif(img_file)
  print(img.has_exif)
  imgexif.artist = f"{versionProgram}"


def writeImage(img, savefilename):
  # A function to allow us to save heif from HDR images with 10bit RGB ( I Think )
  # if its not Apple expect the worst
  cv2.imwrite("/tmp/processed_before.png", img) #before format
  # https://docs.wand-py.org/en/0.4.5/guide/write.html
  # I don't fucking care if devices doesnt support its their problem doesnt support a new tech, while we're on urge to race which one got to the highest first


  # PIL does not support HDR Image, need to load it from Image array format

  register_heif_opener() #specifically to make PIL or pillow able to interact with HEIF or HEIC
  readImg = image.open("/tmp/processed_before.png")
  #readImg = transitionalImgLoad
  readImg.save(savefilename)








#### 1st AL (Abstraction Layer) Wrapper

In [ ]:
#1st layer of wrapper
# Simplifying The routines that need to be done before all the method/function can be used

#------------------------ Simplified Routines Function
#------- Combined Init Routines
#just call one single routines and everything will be executed magically

class trainEmbedEngine_L1():
  def learnImagePattern_L1(shared_memory, prompt):
    global imageGlobalBridge
    imageGlobalBridge=0
    #check whether the model folder and the pt file exists
    # if not do the textual_inversion.py but if it does exit form here and do the generateEngine_L1 with the pipeLoad the pt checkpoint textual inversion
    #launch when it is not available
    print("Launching HuggingFace Textual Inversion!")
    #export MODEL_NAME="duongna/stable-diffusion-v1-4-flax"
    #export DATA_DIR="./cat"
    #os.system(f"python3 ./textual_inversion.py --pretrained_model_name_or_path=$MODEL_NAME--train_data_dir=$DATA_DIR --learnable_property="object" --placeholder_token="<cat-toy>" --initializer_token="toy" --resolution=512 --train_batch_size=1 --max_train_steps=3000 --learning_rate=5.0e-04 --scale_lr --output_dir=textual_inversion_cat ")
    pass

class generateEngine_L1():
  def initT2I_model():
    acceleratorAvailability()
    loadModel2Memory_Text2Img(predefinedModelTarget, predefinedModelUpscalerTarget)

  def initI2I_model():
    acceleratorAvailability()
    loadModel2Memory_Img2Img(predefinedModelTarget, predefinedModelUpscalerTarget)

  def processI2I_wrapper_imagePostProcessing(shared_memory, imgInputPath):
    global seed
    if randomize_seed == 1:
      seed = random.randint(2700000000000,2739896299199)
      logging.debug("[QA Automata Optimization Debug] : Randomized Seed ", seed)
    else:
      seed = globalStaticSeed
    loadModules()
    init_shmem()
    clearModuleState()
    freeUpGlobalMemory()
    global imageGlobalBridge
    imageGlobalBridge=0
    #__________________________________-
    #load image

    leStubPrompt="this is just a stub function to initialize all the variable required"
    textPromptManipulation(leStubPrompt, 1)
    promptHandler(leStubPrompt)
    optimalResolution=reqOptimalResolutionAndModel("", 0) #request Optimal Resolution for the available memory for the post processor of the dE_QARE post processing engine
    #open image using PIL
    imgInput=image.open(imgInputPath).convert('RGB')
    #then invoke post processing with the PIL Raw data and it will output opencv image
    upscalerResolutionCheatingInit(optimalResolution[0], optimalResolution[1])
    openCVOutput=imagePostProcessing(imgInput)
    #we need to convert it to PIL in order write into file'
    nameFile=f"PostProcessingOnly_{seed}.heic"
    writeImage(openCVOutput, nameFile) #the input is opencv format image
    openCVOutput_step1_conversion=cv2.cvtColor(openCVOutput, cv2.COLOR_BGR2RGB)
    print(f"Done {nameFile}")
    #__________________________________-
    shared_memory['result'] = image.fromarray(openCVOutput_step1_conversion)
    clearModuleState()
    freeUpGlobalMemory()

  def processT2I_wrapper_enhanced_embedding_L1(shared_memory, prompt):
    generateEngine_L1.initI2I_model()
    loadModules()
    init_shmem()
    clearModuleState()
    freeUpGlobalMemory()

  def processT2I_wrapper_L1(shared_memory, prompt): #text to image generator interface
    global seed
    if randomize_seed == 1:
      seed = random.randint(2700000000000,2739896299199)
      logging.debug("[QA Automata Optimization Debug] : Randomized Seed ", seed)
    else:
      seed = globalStaticSeed
    print("[QA Automata Diffusion]: Initializing!")
    loadModules()
    init_shmem() #we need global shared memory
    clearModuleState()
    freeUpGlobalMemory()
    #load the global 1st imageLoaded phase to memory or the next cycle
    global imageGlobalBridge
    imageGlobalBridge=0
    prompt = textPromptManipulation(prompt, 1) # doesn't do any manipulation on the prompt globally however it handles if any hyperparameters were inputted through the prompt (prompt, globalMode)
    promptseg=[]
    promptseg=promptHandler(prompt)
    countCycle=0
    totalSegment=0
    #count prompt segment
    if disableMultiPass == 0:
      for count in promptseg:
        totalSegment=totalSegment + 1
      for i in promptseg:
        countCycle = int(countCycle) + 1
        print("[QA Automata Diffusion]: Processing Phase", countCycle, "out of", totalSegment)
        clearModuleState()
        freeUpGlobalMemory()
        if countCycle <= 1:
          if countCycle == totalSegment:
            # = multiprocessing.Process(target= )
            generateImageFromText(i, 0)# 0 indicate enable multicycle processing optimization and upscale the result
            # Explanation on why we need to wrap it using multiprocessing module, Unfortunately we did see a potential where if there are too many prompt it will pile up the VRAM and/or RAM which causes the impossibility to prompt in too many prompt if we didnt wrap it using multiprocessing
            # ugh no you can't do nested Multiprocessing spawn then using CUDA context, it'll fails with async error
            #p = multiprocessing.Process(target=generateImageFromText, args=(i, 0,))
            #p.start()
            #p.join()
            imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
            shared_memory['result'] = imageGlobalBridge
          else:
            i = str(i) + "disablesfx" + "disablequalityenforcement" + "disable_painting_style_enforcement"
            generateImageFromText(i, 1)# 1 indicate enable multicycle processing optimization
            #p = multiprocessing.Process(target=generateImageFromText, args=(i, 1,))
            #p.start()
            #p.join()
            imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
            shared_memory['result'] = imageGlobalBridge
          freeUpGlobalMemory() #cleanup unused memory
        else:
          generateEngine_L1.initI2I_model()
          if countCycle == totalSegment:
            generateImageFromImage(i, 0, imageGlobalBridge) # 0 indicate enable multicycle processing optimization and upscale the result
            #p = multiprocessing.Process(target=generateImageFromImage, args=(i, 0, imageGlobalBridge,))
            #p.start()
            #p.join()
            imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
          else:
            i = str(i) + "disablesfx" + "disablequalityenforcement" + "disable_painting_style_enforcement"
            generateImageFromImage(i, 1, imageGlobalBridge) # 1 indicate enable multicycle processing optimization
            #p = multiprocessing.Process(target=generateImageFromImage, args=(i, 1, imageGlobalBridge,))
            #p.start()
            #p.join()
            imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
            shared_memory['result'] = imageGlobalBridge
        freeUpGlobalMemory()
    else:
      clearModuleState()
      freeUpGlobalMemory()
      generateImageFromText(prompt, 0)# 0 indicate enable multicycle processing optimization
      imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
      shared_memory['result'] = imageGlobalBridge
    #qualityControlGeneratedIMG(prompt) #it will read the last cached preprocessed result of the generated image so you don't have to provide any parameter for the image to the method or function
    freeUpGlobalMemory()

  def processT2I_enhanced_tilingMethod_wrapper_L1(shared_memory, prompt): #text to image generator interface but enhanced using YOLO as the tiling then botch together the image
    # fff1
    global seed
    if randomize_seed == 1:
      seed = random.randint(2700000000000,2739896299199)
      logging.debug("[QA Automata Optimization Debug] : Randomized Seed ", seed)
    else:
      seed = globalStaticSeed
    #------------------------------- STEP 0 ---------------------------------------
    print("[QA Automata Diffusion]: Initializing!")
    print("[QA Automata Diffusion]: Experimental enhanced L1 Wrapper has been called!, use with caution! May cause huge residual Memory usage")
    loadModules()
    init_shmem() #we need global shared memory
    clearModuleState()
    freeUpGlobalMemory()
    #load the global 1st imageLoaded phase to memory or the next cycle
    global imageGlobalBridge
    imageGlobalBridge=0
    prompt = textPromptManipulation(prompt, 1) # doesn't do any manipulation on the prompt globally however it handles if any hyperparameters were inputted through the prompt (prompt, globalMode)
    promptseg=[] #it will split the prompt if its goes beyond the limit
    promptseg=promptHandler(prompt)
    #------------------------------- STEP 1 ---------------------------------------
    # Only generate lowres Image or as the global Canvas
    countCycle=0
    totalSegment=0
    #count prompt segment
    if disableMultiPass == 0:
      for count in promptseg:
        totalSegment=totalSegment + 1
      for i in promptseg:
        countCycle = int(countCycle) + 1
        print("[QA Automata Diffusion]: Processing Phase", countCycle, "out of", totalSegment)
        clearModuleState()
        freeUpGlobalMemory()
        if countCycle <= 1:
          if countCycle == totalSegment:
            # = multiprocessing.Process(target= )
            generateImageFromText(i, 1)# 0 indicate disable multicycle processing optimization and upscale the result
            generateImageFromImage("", 0, imageGlobalBridge) # 0 indicate disable multicycle processing optimization and upscale the result
            # Explanation on why we need to wrap it using multiprocessing module, Unfortunately we did see a potential where if there are too many prompt it will pile up the VRAM and/or RAM which causes the impossibility to prompt in too many prompt if we didnt wrap it using multiprocessing
            # ugh no you can't do nested Multiprocessing spawn then using CUDA context, it'll fails with async error
            shared_memory['result'] = imageGlobalBridge
          else:
            i = str(i) + "disablesfx" + "disablequalityenforcement" + "disable_painting_style_enforcement"
            generateImageFromText(i, 1)# 1 indicate enable multicycle processing optimization
            imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
            shared_memory['result'] = imageGlobalBridge
          freeUpGlobalMemory() #cleanup unused memory
        else:
          generateEngine_L1.initI2I_model()
          if countCycle == totalSegment:
            generateImageFromImage("", 0, imageGlobalBridge) # 0 indicate disable multicycle processing optimization and upscale the result
          else:
            i = str(i) + "disablesfx" + "disablequalityenforcement" + "disable_painting_style_enforcement"
            generateImageFromImage(i, 1, imageGlobalBridge) # 1 indicate enable multicycle processing optimization
            shared_memory['result'] = imageGlobalBridge
        freeUpGlobalMemory()
    else:
      clearModuleState()
      freeUpGlobalMemory()
      generateImageFromText(prompt, 0)# 0 indicate enable multicycle processing optimization
      imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
      shared_memory['result'] = imageGlobalBridge

    #------------------------------- STEP 2 ---------------------------------------
    # detect all the object that are detected (mainly by the YOLO darknet)
    # note : make a c

    #qualityControlGeneratedIMG(prompt) #it will read the last cached preprocessed result of the generated image so you don't have to provide any parameter for the image to the method or function
    freeUpGlobalMemory()

  def processT2I_enhanced_wrapper_L1(shared_memory, prompt): #text to image generator interface but enhanced
    global seed
    if randomize_seed == 1:
      seed = random.randint(2700000000000,2739896299199)
      logging.debug("[QA Automata Optimization Debug] : Randomized Seed ", seed)
    else:
      seed = globalStaticSeed
    print("[QA Automata Diffusion]: Initializing!")
    print("[QA Automata Diffusion]: Experimental enhanced L1 Wrapper has been called!, use with caution! May cause huge residual Memory usage")
    loadModules()
    init_shmem() #we need global shared memory
    clearModuleState()
    freeUpGlobalMemory()
    #load the global 1st imageLoaded phase to memory or the next cycle
    global imageGlobalBridge
    imageGlobalBridge=0
    prompt = textPromptManipulation(prompt, 1) # doesn't do any manipulation on the prompt globally however it handles if any hyperparameters were inputted through the prompt (prompt, globalMode)
    promptseg=[]
    promptseg=promptHandler(prompt)

    #phase0

    countCycle=0
    totalSegment=0
    #count prompt segment
    if disableMultiPass == 0:
      #Phase 1 or Pass 1
      for count in promptseg:
        totalSegment=totalSegment + 1
      for i in promptseg:
        countCycle = int(countCycle) + 1
        print("[QA Automata Diffusion]: Processing Phase", countCycle, "out of", totalSegment)
        clearModuleState()
        freeUpGlobalMemory()
        if countCycle <= 1:
          if countCycle == totalSegment:
            # = multiprocessing.Process(target= )
            generateImageFromText(i, 1)# 0 indicate disable multicycle processing optimization and upscale the result
            generateImageFromImage(i, 0, imageGlobalBridge) # 0 indicate disable multicycle processing optimization and upscale the result
            # Explanation on why we need to wrap it using multiprocessing module, Unfortunately we did see a potential where if there are too many prompt it will pile up the VRAM and/or RAM which causes the impossibility to prompt in too many prompt if we didnt wrap it using multiprocessing
            # ugh no you can't do nested Multiprocessing spawn then using CUDA context, it'll fails with async error
            shared_memory['result'] = imageGlobalBridge
          else:
            i = str(i) + "disablesfx" + "disablequalityenforcement" + "disable_painting_style_enforcement"
            generateImageFromText(i, 1)# 1 indicate enable multicycle processing optimization
            imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
            shared_memory['result'] = imageGlobalBridge
          freeUpGlobalMemory() #cleanup unused memory
        else:
          generateEngine_L1.initI2I_model()
          if countCycle == totalSegment:
            generateImageFromImage(i, 0, imageGlobalBridge) # 0 indicate disable multicycle processing optimization and upscale the result
          else:
            i = str(i) + "disablesfx" + "disablequalityenforcement" + "disable_painting_style_enforcement"
            generateImageFromImage(i, 1, imageGlobalBridge) # 1 indicate enable multicycle processing optimization

      #Phase 2 or Pass 2
      countCycle=0
      totalSegment=0
      for count in promptseg:
        totalSegment=totalSegment + 1

      if totalSegment > 1:
        for i in promptseg:
          countCycle = int(countCycle) + 1
          print("[QA Automata Diffusion]: Processing Phase", countCycle, "out of", totalSegment)
          clearModuleState()
          freeUpGlobalMemory()
          if countCycle <= 1:
            if countCycle == totalSegment:
              # = multiprocessing.Process(target= )
              #generateImageFromText(i, 1)# 0 indicate disable multicycle processing optimization and upscale the result
              generateImageFromImage(i, 0, imageGlobalBridge) # 0 indicate disable multicycle processing optimization and upscale the result
              # Explanation on why we need to wrap it using multiprocessing module, Unfortunately we did see a potential where if there are too many prompt it will pile up the VRAM and/or RAM which causes the impossibility to prompt in too many prompt if we didnt wrap it using multiprocessing
              # ugh no you can't do nested Multiprocessing spawn then using CUDA context, it'll fails with async error
              shared_memory['result'] = imageGlobalBridge
            else:
              i = str(i) + "disablesfx" + "disablequalityenforcement" + "disable_painting_style_enforcement"
              #generateImageFromText(i, 1)# 1 indicate enable multicycle processing optimization
              generateImageFromImage(i, 1, imageGlobalBridge) # 0 indicate disable multicycle processing optimization and upscale the result
              imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
              shared_memory['result'] = imageGlobalBridge
            freeUpGlobalMemory() #cleanup unused memory
          else:
            generateEngine_L1.initI2I_model()
            if countCycle == totalSegment:
              generateImageFromImage(i, 0, imageGlobalBridge) # 0 indicate disable multicycle processing optimization and upscale the result
            else:
              i = str(i) + "disablesfx" + "disablequalityenforcement" + "disable_painting_style_enforcement"
              generateImageFromImage(i, 1, imageGlobalBridge) # 1 indicate enable multicycle processing optimization

      shared_memory['result'] = imageGlobalBridge
      freeUpGlobalMemory()
    else:
      clearModuleState()
      freeUpGlobalMemory()
      generateImageFromText(prompt, 0)# 0 indicate enable multicycle processing optimization
      imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
      shared_memory['result'] = imageGlobalBridge
    #qualityControlGeneratedIMG(prompt) #it will read the last cached preprocessed result of the generated image so you don't have to provide any parameter for the image to the method or function
    freeUpGlobalMemory()


  def processI2I_wrapper_L1(shared_memory, prompt, img): #image to image generator interface
    global seed
    if randomize_seed == 1:
      seed = random.randint(2700000000000,2739896299199)
      logging.debug("[QA Automata Optimization Debug] : Randomized Seed ", seed)
    else:
      seed = globalStaticSeed
    print("[QA Automata Diffusion]: Initializing!")
    loadModules()
    init_shmem() #we need global shared memory
    clearModuleState()
    freeUpGlobalMemory()
    #load the global 1st imageLoaded phase to memory or the next cycle
    global imageGlobalBridge
    imageGlobalBridge=0
    prompt = textPromptManipulation(prompt, 1) # doesn't do any manipulation on the prompt globally however it handles if any hyperparameters were inputted through the prompt (prompt, globalMode)
    promptseg=[]
    promptseg=promptHandler(prompt)
    countCycle=0
    totalSegment=0
    imgInput=image.open(img).convert('RGB')
    #count prompt segment
    if disableMultiPass == 0:
      for count in promptseg:
        totalSegment=totalSegment + 1
      for i in promptseg:
        countCycle = int(countCycle) + 1
        print("[QA Automata Diffusion]: Processing Phase", countCycle, "out of", totalSegment)
        clearModuleState()
        freeUpGlobalMemory()
        if countCycle <= 1:
          if countCycle == totalSegment:
            generateImageFromImage(i, 0, imgInput)# 0 indicate enable multicycle processing optimization and upscale the result
            #p = multiprocessing.Process(target=generateImageFromImage, args=(i, 0, imgInput,))
            #p.start()
            #p.join()
            #imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable

          else:
            i = str(i) + "disablesfx" + "disablequalityenforcement" + "disable_painting_styenforcement"
            generateImageFromImage(i, 1, imgInput)# 1 indicate enable multicycle processing optimization
            #p = multiprocessing.Process(target=generateImageFromImage, args=(i, 1, imgInput,))
            #p.start()
            #p.join()
            #imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
            shared_memory['result'] = imageGlobalBridge
          freeUpGlobalMemory() #cleanup unused memory
        else:
          if countCycle == totalSegment:
            generateImageFromImage(i, 0, imageGlobalBridge) # 0 indicate enable multicycle processing optimization and upscale the result
            #p = multiprocessing.Process(target=generateImageFromImage, args=(i, 0, imageGlobalBridge,))
            #p.start()
            #p.join()
            #imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
          else:
            i = str(i) + "disablesfx" + "disablequalityenforcement" + "disable_painting_style_enforcement"
            generateImageFromImage(i, 1, imageGlobalBridge) # 1 indicate enable multicycle processing optimization
            #p = multiprocessing.Process(target=generateImageFromImage, args=(i, 1, imageGlobalBridge,))
            #p.start()
            #p.join()
            #imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
            shared_memory['result'] = imageGlobalBridge
        shared_memory['result'] = imageGlobalBridge
        freeUpGlobalMemory()
    else:
      clearModuleState()
      freeUpGlobalMemory()
      generateImageFromText(prompt, 0)# 0 indicate enable multicycle processing optimization
      imageGlobalBridge=imageGlobalBridge # bridging the result from the multiprocess Global Variable
      shared_memory['result'] = imageGlobalBridge
    #qualityControlGeneratedIMG(prompt) #it will read the last cached preprocessed result of the generated image so you don't have to provide any parameter for the image to the method or function
    freeUpGlobalMemory()



#### 2nd AL (Abstraction Layer) Wrapper

In [ ]:
#2nd layer of Wrapper ( multiprocessing memory wrapper )
#Text to Image
#WARNING : Any traceback won't be written to the console but rather it just going to kill itself!
class trainEmbedEngine_L2():
  pass



class generateEngine_L2():
  def processT2I(prompt):
    init_shmem()
    p = multiprocessing.Process(target=generateEngine_L1.processT2I_wrapper_L1, args=(shared_memory, prompt,))
    p.start()
    p.join()
    #return readable PIL Image from the result
    return shared_memory['result']

  def processT2I_enhanced(prompt):
    init_shmem()
    p = multiprocessing.Process(target=generateEngine_L1.processT2I_enhanced_wrapper_L1, args=(shared_memory, prompt,))
    p.start()
    p.join()
    #return readable PIL Image from the result
    return shared_memory['result']

  # Image to Image
  def processI2I(prompt, img):
    init_shmem()
    p = multiprocessing.Process(target=generateEngine_L1.processI2I_wrapper_L1, args=(shared_memory, prompt, img,))
    p.start()
    p.join()
    #return readable PIL Image from the result by utilizing shared_memory variable which initialized globally
    return shared_memory['result']

  def postProcessI2I(imgInputPath):
    init_shmem()
    #processI2I_wrapper_imagePostProcessing
    p = multiprocessing.Process(target=generateEngine_L1.processI2I_wrapper_imagePostProcessing, args=(shared_memory, imgInputPath,))
    p.start()
    p.join()
    #return readable PIL Image from the result by utilizing shared_memory variable which initialized globally
    return shared_memory['result']

#by using multiprocessing Process, it allows us to remove almost completely the memory from work memory after its done
# Special thanks to this forum post : https://stackoverflow.com/questions/51005147/keras-release-memory-after-finish-training-process


#### 3rd AL (Abstraction Layer) Wrapper
> This usually for automation for another function that don't need to control much of the internal (Like the multiprocessing part) but rather require multiple steps of the image processing

In [ ]:
class generateEngine_L3():
  def processIMGRestoration(img):
    pass
    #1st pass pix2pix process
    lePrompt = "disable_painting_style_enforcement, disablesfx, disable_quality_fence, forcePix2PixMode, remove blur, preserve details, warm cinematic color grading, add sharpness, remove lensflare"



## Additional Plugins for Applications on Bussiness

##### T-Shirt Merch Function Adaptation from Willy Code

In [ ]:
#ONLY FOR QUESTANDACHIEVEMENT BUSINESS
# Business Plugin (Code style may not coherent with other part of the code)

class projectDaredevilMerchGen_L2():
  def generateTshirt(prompt):
    init_shmem() #we need global shared memory
    imageGenerated = generateEngine_L2.processT2I(prompt)
    output_filename = "daredevil_CognoCouture2_Experimental.jpg"
    p = multiprocessing.Process(target=projectDaredevilMerchGen_L1.img_to_tshirt, args=(shared_memory, imageGenerated, output_filename,))
    p.start()
    p.join()


class projectDaredevilMerchGen_L1():
  def img_to_tshirt(shared_memory, imageInputData, output_filename):
    moduleName = "[WillyMerchGen]: "
    print(moduleName + "Written by Willy")
    print(imageInputData)
    """Produce Q7 Merch Clothes from generated Q7AIE Diffusion

    Parameters
    ----------
    image_path : input the image path here, .png or .jpg supported
    output_filename : desired output filename (with file extension)

    Returns
    ----------
    Log (string) that result image is saved to [location]

    Examples
    ----------
    >>> waifu_to_merch_clothes("input.png", "output.png")
    'Result image saved to: ./merch/output.png'

    """
    print(moduleName + "Loading Python Module")
    loadModules()
    clearModuleState()
    freeUpGlobalMemory()
    print(moduleName + "Requesting Assets Extraction Module")
    specialProjectDaredevil.merchSpriteAssetsExtraction() #will output fullFrontRearTShirtTemplate.jpg and other upcoming Template for future Sprite
    requestSprites() #if q7Logo.png sprites and other png are needed

    print(moduleName + "Inverting Logo")
    # Logo need to be inverted due to the logo is white whilst the shirt is white and rip contrasst
    #invert the image color
    logo = Image.open("q7Logo.png")
    r, g, b, a = logo.split()
    # Invert the RGB channels
    r = imageops.invert(r)
    g = imageops.invert(g)
    b = imageops.invert(b)
    # Recombine the inverted RGB channels with the original alpha channel
    logo_inv = Image.merge('RGBA', (r, g, b, a))
    logo_inv.save("logo_inverted.png")

    #----------------------------
    # This is the process for Images that getting slapped into the shirt
    # Getting Resolution and Maximum coordinate phase
    print(moduleName + "Albert Modification, Preloading all Image ")
    logo_inv_w, logo_inv_h = logo_inv.size
    print(moduleName + "Albert Modification, Logo watermark resolution is " + str(logo_inv_w) + "," + str(logo_inv_h))
    imgIllustGenerated = imageInputData
    imgIllustGenerated_w, imgIllustGenerated_h = imgIllustGenerated.size
    print(moduleName + "Albert Modification, Image Illustration resolution is " + str(imgIllustGenerated_w) + "," + str(imgIllustGenerated_h))
    print(moduleName + "Albert Modification, Getting Max Coordinate base canvas or the Tshirt Template")
    bg = Image.open("./fullFrontRearTShirtTemplate.jpg")
    new_merch = bg
    bg_w, bg_h = new_merch.size
    print(moduleName + "Albert Modification, Canvas Max Coordinate" + str(bg_h) + "," + str(bg_w))


    # Defining what is the anchor point or comparison point for rescaling and/or positioning sprites
    if bg_h > bg_w: #for scaling were going to use the width or the height of the canvas or the tshirt template to determine how big should we slap the logo into
      comparisonPointScale = bg_w
    else:
      comparisonPointScale = bg_h

    #Resizing Phase ----------------------

    # Rescaling Logo that will be on the shirt
    print(moduleName + "Albert Modification, Resizing Logo")
    scalePercentage = 6 # scale percentage based on how big is the comparison scale is
    resTarget = int(round(((scalePercentage/100)*comparisonPointScale)))
    aspectRatioImage = (logo_inv_w/logo_inv_h)
    #just to make sure if the logo weren't 1:1 and and the function is quite modular
    if logo_inv_h > logo_inv_w: # h=y (height) w=x (width)
      target_y = int(resTarget)
      target_x = int(round(target_y*aspectRatioImage))

    else:
      target_x = int(resTarget)
      target_y = int(round(target_x*aspectRatioImage))
    logo_inv = logo_inv.resize((target_x, target_y), Image.ANTIALIAS)

    #Rescaling Generated Image

    print(moduleName + "Albert Modification, Resizing Generated Image")
    scalePercentage = 21 # scale percentage based on how big is the comparison scale (canvas or the template) is
    resTarget = int(round(((scalePercentage/100)*comparisonPointScale)))
    aspectRatioImage = (imgIllustGenerated_w/imgIllustGenerated_h)
    #just to make sure if the logo weren't 1:1 and and the function is quite modular
    if imgIllustGenerated_h > imgIllustGenerated_w: # h=y (height) w=x (width)
      target_y = int(resTarget)
      target_x = int(round(target_y*aspectRatioImage))
      print("debugPortrait" + str(imgIllustGenerated_h) + "," + str(imgIllustGenerated_w))
    else:
      target_x = int(resTarget)
      target_y = int(round(target_x*aspectRatioImage))
      print("debugLandscape")
    imgIllustGenerated = imgIllustGenerated.resize((target_x, target_y), Image.ANTIALIAS)


    #Position Determining Phase ----------------
    #this has to be specifically tuned! for each canvas or shirt template
    #
    # the max coordinates are bg_w(x),bg_h(y) (maximum x is on the right side and maximum y is on the bottom)
    #Logo
    spriteSize_x, spriteSize_y = logo_inv.size
    posPercentage_x = 70.85
    posPercentage_y = 23
    posTarget_sprite_logo_x = int(round((posPercentage_x/100)*bg_w))
    posTarget_sprite_logo_y = int(round((posPercentage_y/100)*bg_h))

    #Generated Image
    # Portrait and Landscape mode need to be splitted, due to the fact they're different configuration on the canvas

    spriteSize_x, spriteSize_y = imgIllustGenerated.size
    posPercentage_x = 19.5
    posPercentage_y = 32
    posTarget_sprite_imgGenerated_x = int(round((posPercentage_x/100)*bg_w))
    posTarget_sprite_imgGenerated_y = int(round((posPercentage_y/100)*bg_h))


    print(moduleName + "Resizing Generated Image")
    new_img = imgIllustGenerated
    print(moduleName + "Inserting Background Image T-shirt")
    # Combine image with the Template shirt (shirt as the background)
    new_merch.paste(new_img, (posTarget_sprite_imgGenerated_x, posTarget_sprite_imgGenerated_y))
    new_merch.paste(logo_inv, (posTarget_sprite_logo_x, posTarget_sprite_logo_y), logo_inv)
    #??? Test
    #new_merch.paste(logo_inv, (4828, 2820), logo_inv) #for the canvas it was 4828x2820 however idk why its not even aligned for the max coordinate
    # oh well we can just insert the scale beyond 100%

    # save output
    print(moduleName + "Saving to shared_memory!")
    shared_memory['merch_tshirt_result'] = new_merch
    new_merch.save(output_filename)


    print(moduleName + "Cleaning!")
    # cleaning up
    os.remove("logo_inverted.png")

    return f"Result image saved to: {output_filename}"



#### Function Test

In [ ]:
#init_shmem() #we need global shared memory
#generateEngine_L1.processT2I_enhanced_wrapper_L1(shared_memory,"1girl, Short sweet rounded teen girl, childlike face, small chest, casual fully enclosed pink white girl clothing, short pink colored hair")

## Launch Program (Simple GUI)

#### Text to Image Prompt

#### Image to Image GUI

In [ ]:
#https://github.com/zauberzeug/nicegui

## Interact the program with writing or calling the function

> processT2I(yourprompt)

``` also for the files will be written (working directory) and can be downloaded through the file explorer of this web interface ```


In [ ]:
#processT2I("landscape, shining day, Sunflower Garden Village, crowd in the colorful garden, night sky, falling star, clear fish pond")
#processT2I("landscape, Rose Garden, Garden Village Background, in the colorful garden, clear fish pond, crowd, cloudy sky, elegant castle background, flying air baloon, disney, flying birds")
#processT2I("1girl, teen short pink colored hair, winter")


#_Sangreverde Sisters_ but design only focuses on the Character features so later on at 0.7.0 Release when textual inversion introduces we able to combine them and make them do something in action
#A fusion of Spanish words "sangre" (blood) and "verde" (green), this name signifies the intertwining of life force and nature. It alludes to the sisters' connection to the earth and the unseen forces that bind them.
# Vermella Blaze
#lePrompt = "disable_painting_style_enforcement. forcePortraitMode, 1girl, short black hair, half rim glasses, red eyes, sweet soft Tsundere girl, adult, serious confidence, round cute face medium chest, long black enclosed coat cape, dark night"
# Aemeraldine Serenella
#lePrompt = "disable_painting_style_enforcement, 1girl, Long slightly wavy black hair, half rim round glasses, wide body, green eyes, mysterious sweet soft kuudere girl, adult, shy blushing, round cute face medium chest, long white elegant coat green cape, night sky, 0 degree shot"
#Mella and Nella xd

#New IP?
# Anne Dottir
#lePrompt = "disable_painting_style_enforcement. forcePortraitMode, 1girl, Long black hair, half rim glasses, red eyes, sweet soft Yandere girl, adult, serious shy blushing, round cute face medium chest, long black elegant coat cape, mysterious, night sky, abandoned station"
#lePrompt = "disable_painting_style_enforcement. forcePortraitMode, 1girl, Long black hair, half rim glasses, red eyes, sweet soft Yandere girl, adult, serious confidence, round cute face medium chest, long black elegant coat cape, mysterious, night sky"


#Amara Celeste?
#lePrompt = "disable_painting_style_enforcement, 1girl, Long slightly wavy black hair, half rim glasses, green eyes, sweet soft kuudere girl, adult, shy blushing, round cute face medium chest, long black elegant coat cape, mysterious, night sky, Drinking green Ocha tea, House"
#lePrompt = "disable_painting_style_enforcement, 1girl, Long slightly wavy black hair, half rim glasses, green eyes, sweet soft kuudere girl, adult, shy blushing, round cute face medium chest, long black elegant coat cape, mysterious, night sky, Walking on forest lantern"
#lePrompt = "disable_painting_style_enforcement, 1girl, Long slightly wavy black hair, half rim glasses, green eyes, sweet soft kuudere girl, adult, shy blushing, round cute face medium chest, long black elegant coat cape, mysterious, night sky, Aurora"

#lePrompt = "disable_painting_style_enforcement, 1girl, Long slightly wavy black hair, half rim glasses, green eyes, sweet soft kuudere girl, chubby adult, grin shy blushing, round cute face medium chest, long black elegant coat cape, mysterious, night sky, abandoned station"
#lePrompt = "disable_painting_style_enforcement, 1girl, Long black hair, half rim glasses, green eyes, sweet soft kuudere girl, adult, shy blushing, round cute face medium chest, Casual Dress, mysterious, night sky, cuddle, abandoned station"

#lePrompt = "1girl, Long black hair, half rim glasses, green eyes, sweet soft kuudere girl, adult, shy blushing, round cute face medium chest, Casual summer clothes, straw hat, mysterious, grinning, Seashore, seagulls flying"
#lePrompt = "1girl, Long black hair, half rim glasses, green eyes, sweet soft kuudere girl, adult, shy blushing, round cute face medium chest, long black elegant coat cape, mysterious, Epic and Legendary Fighting, Archer Aiming, monster hunter"
#lePrompt = "1girl, Long black hair, half rim glasses, green eyes, sweet soft kuudere girl, adult, shy, round cute face medium chest, long red elegant coat cape, mysterious, Epic and Legendary Fighting, Archer Aiming, monster hunter"
#lePrompt = "disable_painting_style_enforcement, 1girl, Long black hair, half rim glasses, green eyes, sweet soft kuudere girl, adult, shy, round cute face medium chest, long red elegant coat cape, mysterious, Delving into the ruins, intrigued by artifacts"
#lePrompt = "disable_painting_style_enforcement, 1girl, Long black hair, half rim glasses, green eyes, sweet soft kuudere girl, adult shy, cry tear joy, round cute face medium chest, long white wedding Clothes, bride, mysterious, night sky, Town Village"
#lePrompt = "disable_painting_style_enforcement, 1girl, Long black hair, half rim glasses, green eyes, sweet soft kuudere girl, confidence, adult, shy smile, round cute face medium chest, long red elegant coat cape, mysterious, high winds"


#lePrompt = "disable_painting_style_enforcement, disablesfx, pixivArt, 1boy, medium black chocolate hair, Long coat dressed, adult shy boy, round face, lantern"
#lePrompt = "disable_painting_style_enforcement, disablesfx, disablequalityenforcement, 1boy, petruk"

#Req Ninja Mask Petruk, belt batik (Bg chr: 1500, Europe Colonization, Interact fluently with the colonizer Netherland, Fountain of Youth, )
#lePrompt = "disable_painting_style_enforcement, forceLandscapeMode, 1boy, Teen Boy, Medium white hair, red eyes, Masked, Batik Belt, samurai, Europe Night Colonization, Middle age era"



#ofc we also need to create the opposite of the parts
#processT2I("1boy, standard boy, Short chocolate hair, adult cute, medium coat cape, training on base with bow")

#Stella
#lePrompt = "disable_painting_style_enforcement, 1girl, Short haired golden brownish, golden eyes, Tsundere adult girl, shy, serious, medium height, elegant white-gold long coat, black gloves, wide cyborg body"
#lePrompt = "disable_painting_style_enforcement, 1girl, Short haired golden brownish, golden eyes, Tsundere adult girl, shy, serious, medium height, short skirt, casual white gold jacket, black gloves, wide robotic body"


#lePrompt = "disable_painting_style_enforcement, Mika Pikazo artstyle, 1girl, Short sweet adult girl, wide body, shy grin, medium chest, red color backpack, pink dress, long pink hair, garden, butterfly"
##lePrompt = "1girl, Short sweet rounded teen girl, childlike face, medium chest, casual fully enclosed pink white girl clothing, short pink colored hair, near the shore"
#lePrompt = "1girl, Short sweet rounded teen girl, teenage face, medium chest, casual fully enclosed pink white girl clothing, short pink colored hair, near seashore, straw hat"

#lePrompt = "katana, Samurai Soldier Apparel, final fantasy lightning, Dark Blue, red short haired yandere adult girl, Long scifi sharp blade, legendary, crowd soldier background, final fantasy background landscape"
#processT2I("1girl, Elysia, Honkai Impact, Elf, Maid, wolf cut hair, pink hair, medium chest")

#lePrompt = "1girl, Sucrose, Genshin Impact, casual princess summer clothes, scientist, long green haired girl, short girl, half round rim glasses, Lowered fox animal ears, cute shy kuudere, grinning, Seashore, seagulls flying"
#lePrompt = "1girl, Medium chest, short hair, bob cut, black and yellow shadow hair, teen girl, retrowave future storm crowd background, scifi epic long elegant coat, yellow eye, black red dress"
#processT2I("Simple Sketch, Simple House on middle of field, Draft")
#processT2I("disable_painting_style_enforcement, Warrior, Epic Battles, Monster Hunter, Big kingdom, fantasy, apocalypse, armored dragon, aurora blue sky, falling stars, shattered sky, daredevil, explosion background")
#lePrompt = "disable_painting_style_enforcement, disablesfx, Warrior, Epic Battles, Monster Hunter, Big kingdom, fantasy, apocalypse, armored dragon, aurora blue sky, falling stars, shattered sky, daredevil, explosion background"
#lePrompt = "disable_painting_style_enforcement, forceLandscapeMode, 1boy, Fantasy warrior battles an armored dragon amidst an apocalypse. Sky is aurora blue with falling stars and a shattered backdrop. Explosive battle scene with daredevil monster hunter"
#processT2I("Space, background, Saturn, cogwheel planet rings, planets revolves around saturn, starry sky, shooting stars, Flying pirate ship")
#lePrompt = "1girl")
#processT2I("Portrait, Cassandra, Rainbow Aurora Sky, Stary Night, starfall, flower field, shattered night")


#cyberpunk
#Test
#lePrompt = "girl, disable_painting_style_enforcement, forceLandscapeMode, Town, Dystopian, Cyberpunk, Landscape, Raining, Falling Stars"

# Landscape House Painting
#processT2I("disable_painting_style_enforcement, watercolor, canvas splash, landscape, Rose Garden, Garden Village Background, in the colorful garden, clear fish pond, crowd, crystal sky, elegant crystal castle background, disney")
#lePrompt = "Landscape, isolated, Radio on the middle field, middle of flower field, butterflies"
#lePrompt = "disable_painting_style_enforcement, Minecraft, Steve adventure, Blocky world"

#lePrompt = "Space, Dual color art, Saturn cogwheel planet rings, planets revolves around saturn, shooting stars, inside Flying space train, butterflies, flying whales, Morning star sun rays, steampunk"
#lePrompt = "Space, Saturn cogwheel planet rings, planet saturn, shooting stars, Flying Blue White Gundam robot, Morning star sun rays, Crystal Moon"
#lePrompt = "Cheerful Astrounauts, Drinking Glass Tea, elegant, aesthethic space cafe, Space, Saturn cogwheel planet rings, planet saturn, shooting stars, Flying Blue White Gundam, Morning star sun rays, Crystal Moon, Sea blue Sky"

#lePrompt = "disable_painting_style_enforcement, Cheerful Astrounauts, Drinking Glass Tea, elegant, black gold aesthethic space cafe, Saturn cogwheel planet rings, planet saturn, Morning star sun rays, Crystal Moon, Sea blue Sky, indoor"


#lePrompt = "Morning Light rays, Marketplace, Colorful Japanese Town, Bridge, Night, Garden Village, shooting stars, Candle Light, steampunk"
#lePrompt = "indoor, Morning Light Rays, Dust lightrays, abandoned room, Elegant, Laboratory Equipment, Computer Desktop, Laptop on Table, Dust light rays, small flower pot, carpet in the middle, bird on windows"
#lePrompt = "Space, Dual color art, Astronauts, Smiling, Saturn cogwheel planet rings, planets revolves around saturn, shooting stars, Flying space train, butterflies, flying whales, Morning star sun rays, aesthethic black golden tea cafe, steampunk"
#lePrompt = "Morning Sky, Cloud city kingdom, above cityscape, Dropping from sky, Falling stars, Raining, Wet Windows, Apartement"

#lePrompt = "disable_painting_style_enforcement, forceLandscapeMode, Space, Smiling, planets revolves around saturn, starry sky, Flying railways, space train railway, colorful flowers, Space labs"
#ratioOverride=[8, 4]
#lePrompt = "disable_painting_style_enforcement, Airplane Fighter Scifi Jet, Zooming through sky, Over the urban areas, Cloudy"
#lePrompt = "disablequalityenforcement, disablesfx, disable_painting_style_enforcement, water splash art, white canvas, revealing unopened treasure on the canvas"
#lePrompt = "disablequalityenforcement, disablesfx, disable_painting_style_enforcement, toon, simple Green Crystal half cogwheel, Simple Design, material style, White Background"


#lePrompt = "disablesfx, disable_painting_style_enforcement, forceBalanceMode, cat, Walking cat, Village Background"
#lePrompt = "disablequalityenforcement, disablesfx, disable_painting_style_enforcement, Earth Atmosphere Exosphere, Earth Curvature, Shooting Stars sky, Earth Transition to Space, cloudy sky, sea blue sky, Floating Castle, Flying Kid with wings"
#lePrompt = "disablequalityenforcement, disablesfx, disable_painting_style_enforcement, Tiny, Japanese, Lovable, Pastel Colors, forceVectorMode, small character, white background, Simple, no-gradient, no-text"


#lePrompt = "disable_painting_style_enforcement, disablesfx, forceLandscapeMode, Night Landscape, One mountains, pure nature landscape, near the shore"

#lePrompt = "disable_painting_style_enforcement, 1boy, teen, walking on abandoned land, near shore, close to the camera, godrays, Unknown ruins"
#ratioOverride=[5, 9]

#lePrompt = "disable_painting_style_enforcement, disablesfx, forceLandscapeMode, Workroom, Turqoise Colored Table, Desklamp, Bunch of Research Papers, Pen"
#ratioOverride=[16, 9]
#lePrompt = "disable_painting_style_enforcement, 1boy, wide body, round cute face, shy grin, Chocolate medium hair, blue iris eyes, chocolate long elegant coat, bowtie, Grin at Camera, Waving Hi, Close to Camera"

#lePrompt = "disable_painting_style_enforcement, 1girl, Onodera, Nisekoi, hajidere, medium chest, medium height, black medium haired, black chocolate eyes, long elegant coat, shy blushing"
#lePrompt = "disable_painting_style_enforcement, 1girl, cute adult girl, wide body, shy grin, half rim glasses, long Wavy Hair, Hair Bowtie, Reimu Hakurei, Red Shrine Maiden, long white coat, shrine"
#lePrompt = "disable_painting_style_enforcement, 1girl, cute adult girl, shy grin, Marisa Kirisame, Half Rimmed Glasses, Golden Long Wavy Hair, Witch Hat, wide body, Sitting bench, long skirt, Cherry Blossom"

#lePrompt = "disable_painting_style_enforcement, forcePortraitMode, 1girl, Itsuki Nakano, Gottobun, Hajidere, medium chest, high height, Red Wavy Long Hair, Star hairpin, Smirk, Long Dress"
lePrompt = "disable_painting_style_enforcement, forcePortraitMode, 1girl, Princess, Primary School, shy, Medium Chest, Medium height, brown Short wavy Hair, Giving Gift, White Fountain Background, Sea of Stars Sky, Night sky"
#lePrompt = "disable_painting_style_enforcement, forceRealismMode, Architect Mode, Mansion, small flower Park infront of entrance, arabic style, Glass building"


#lePrompt = "disable_painting_style_enforcement, disablesfx, forceLandscapeMode, looking at the sky, 1boy, grin happy, using helmet, chocolate hair, blue eyes, float, Space, Astronaut, Sea of stars, casual, space"
#ratioOverride=[27, 4]

#lePrompt = "disable_painting_style_enforcement, disablesfx, forceLandscapeMode, looking at the sky, 1boy, grin happy, using helmet, chocolate hair, blue eyes, float, Space, Astronaut, Sea of stars, Space Suit"
#ratioOverride=[16, 9]
#lePrompt = "disable_painting_style_enforcement, disablesfx, forceLandscapeMode, 1boy, scientist boy , happy, serious, using helmet, Space, Astronaut, Sea of stars, Space Suit"
#lePrompt = "disable_painting_style_enforcement, forceLandscapeMode, 1cat"

#lePrompt = "disable_painting_style_enforcement, disablesfx, forcePortraitMode, cat, feline, licking hand, middle of village"
#lePrompt = "disable_painting_style_enforcement, disablesfx, forceLandscapeMode, crowd, serenity, Raining Night lights, Japanese Lantern Village, Steampunk Village"
#ratioOverride=[4.12415, 8.412] #you can comment this variable if you dont want to override the resolution output
#ratioOverride=[8.412, 4.12415] #you can comment this variable if you dont want to override the resolution output
#ratioOverride=[16, 9]
#ratioOverride=[9, 16]
ratioOverride=[6, 14]

#lePrompt = "disable_painting_style_enforcement, disablesfx, forceRealismMode, Alpaca, Animal, Smile, white, near shore"
#lePrompt = "disable_painting_style_enforcement, disablesfx, 1girl, star night, forcePortraitMode, white hat, mascot, Zephyrine, White dressed, short ginger hair, portrait, Medium body, grin, cute, green eyes"
#lePrompt = "disable_painting_style_enforcement, disablesfx, 1girl, star night, forcePortraitMode, white hat, mascot, Zephyrine, White dressed, short ginger hair, portrait, Medium body, Reading book, serious"
#Zephyrine Emotion

#icon
#lePrompt = "disable_painting_style_enforcement, disablesfx, 1girl, star night, forcePortraitMode, white hat, mascot, Zephyrine, White  long coat, long ginger hair, portrait, Medium body,cute, green eyes, grin"

"""Happiness: A positive emotional state characterized by feelings of joy, contentment, and satisfaction.

Sadness: A negative emotional state characterized by feelings of unhappiness, sorrow, and grief.

Fear: An emotional response to perceived threats or danger, triggering a "fight or flight" response to protect oneself.

Anger: A strong emotional reaction to perceived frustration, injustice, or provocation, often involving feelings of irritation, annoyance, or rage.

Disgust: An emotional response to offensive, repulsive, or undesirable stimuli, leading to avoidance or withdrawal."""#
#lePrompt = "disable_painting_style_enforcement, disablesfx, 1girl, star night, forcePortraitMode, white hat, mascot, Zephyrine, White long coat, long ginger hair, portrait, Medium body, cute, green eyes, Smile Excited"
#lePrompt = "disable_painting_style_enforcement, disablesfx, 1girl, star night, forcePortraitMode, white hat, mascot, Zephyrine, White  long coat, long ginger hair, portrait, Medium body,cute, green eyes, disappointed pout"
#lePrompt = "disable_painting_style_enforcement, disablesfx, 1girl, star night, forcePortraitMode, white hat, mascot, Zephyrine, White  long coat, long ginger hair, portrait, Medium body,cute, green eyes, Sad Cry Tears"
#lePrompt = "disable_painting_style_enforcement, disablesfx, 1girl, star night, forcePortraitMode, white hat, mascot, Zephyrine, White  long coat, long ginger hair, portrait, Medium body, cute, green eyes, Angry Angst Frustration"
#lePrompt = "disable_painting_style_enforcement, disablesfx, 1girl, star night, forcePortraitMode, white hat, mascot, Zephyrine, White  long coat, long ginger hair, portrait, Medium body, cute, green eyes, Angry"
#lePrompt = "disable_painting_style_enforcement, disablesfx, 1girl, star night, forcePortraitMode, white hat, mascot, Zephyrine, White long coat, long ginger hair, portrait, Medium body, cute, green Closed eyes, concerned Blush Scared"

#---------------
#lePrompt = "disable_painting_style_enforcement, forceLandscapeMode, disablesfx, prince boy, white green long coat, cute, mascot, short white hair, monocle, grey hat, grin, night, heartwarming, majestic"
#lePrompt = "disable_painting_style_enforcement, forceLogoMode, disablesfx, disable_painting_style_enforcement, disable_quality_fence, Letter Z, Blue Flat logo, Gradient Blue Round Logo"
#lePrompt = "disable_painting_style_enforcement, disablesfx, forceLandscapeMode, Alpaca, Gold Crown, long blue coat, Animal, white"
#ratioOverride=[8, 8]
#ratioOverride=[8, 13]

# however take a note that if you are forcing to use a specific model you need to override the ratio
saveImageDump = generateEngine_L2.processT2I_enhanced(lePrompt)
#saveImageDump = generateEngine_L2.processT2I(lePrompt)

[QA Automata Diffusion]: Initializing!
[QA Automata Diffusion]: Experimental enhanced L1 Wrapper has been called!, use with caution! May cause huge residual Memory usage
ForceModeModel  1
[QA Automata Diffusion]: Processing Phase 1 out of 1
[QA Automata Optimization Debug] : 1
[QA Automata Optimization Debug] : The Processor Unit Only Capable Rendering at  416 * 632


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


  0%|          | 0/40 [00:00<?, ?it/s]

[QA Automata Optimization Debug] : 1
[QA Automata Optimization Debug] : The Processor Unit Only Capable Rendering at  936 * 1424


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

[QA Automata Diffusion] : Saving File, (This might take a while)  ./dE_QARE_0.6.5_Beta_2703843720886_2023-10-11 08:40:10.378827.heic
[QA Automata Diffusion] : If your device mostly Android, can't open this picture try other device that is not Android
[QA Automata Diffusion] : We won't provide any compatibility to device that can't open the Picture, blame your vendor and developer to not support such open standard


<ipython-input-39-f3870734bf5c>:944: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  imageRescaledForShitGradio = images_postProcessed.resize((round((optimalResolution[0])/optimalResolution[4]), round((optimalResolution[1])/optimalResolution[4])), image.ANTIALIAS)


[QA Automata Optimization Critical Warning] : Can't fit model into the Memory! 
[QA Automata Optimization Critical Warning] : Can't fit model into the Memory! 
[QA Automata Optimization Critical Warning] : Can't fit model into the Memory! 


In [ ]:
"""lePrompt = "disable_painting_style_enforcement, disablesfx, forceLandscapeMode, 1cat, calico cat, relaxing bed, big room, Butterflies"
ratioOverride=[4.12415, 8.412]
saveImageDump = generateEngine_L2.processI2I(lePrompt, "./Oni.jpg")"""

'lePrompt = "disable_painting_style_enforcement, disablesfx, forceLandscapeMode, 1cat, calico cat, relaxing bed, big room, Butterflies"\nratioOverride=[4.12415, 8.412]\nsaveImageDump = generateEngine_L2.processI2I(lePrompt, "./Oni.jpg")'

In [ ]:
"""#Image Enhancing Proccessor Pix2Pix Restoration
lePrompt = "disable_painting_style_enforcement, disablesfx, disable_quality_fence, forcePix2PixMode, remove blur, preserve original details, High resolution, warm cinematic color grading, add sharpness"
ratioOverride=[16, 9]
saveImageDump = generateEngine_L2.processI2I(lePrompt, "./f619419368.jpg")
#saveImageDump = generateEngine_L2.processI2I(lePrompt, "./f633447408.jpg")"""

#what if only upscaling and other color correction?
#saveImageDump = generateEngine_L2.postProcessI2I("./f559847840.jpg")

'#Image Enhancing Proccessor Pix2Pix Restoration\nlePrompt = "disable_painting_style_enforcement, disablesfx, disable_quality_fence, forcePix2PixMode, remove blur, preserve original details, High resolution, warm cinematic color grading, add sharpness"\nratioOverride=[16, 9]\nsaveImageDump = generateEngine_L2.processI2I(lePrompt, "./f619419368.jpg")\n#saveImageDump = generateEngine_L2.processI2I(lePrompt, "./f633447408.jpg")'

In [ ]:
"""def listPhotos(directory):
    jpg_files = glob.glob(os.path.join(directory, "*.jpg"))
    return jpg_files
Restoredirectory="./restorePhot"
jpg_photos = listPhotos(Restoredirectory)
for jpg_photo in jpg_photos:
  print(jpg_photo)
  saveImageDump = generateEngine_L2.postProcessI2I(f"{jpg_photo}")

"""

'def listPhotos(directory):\n    jpg_files = glob.glob(os.path.join(directory, "*.jpg"))\n    return jpg_files\nRestoredirectory="./restorePhot"\njpg_photos = listPhotos(Restoredirectory)\nfor jpg_photo in jpg_photos:\n  print(jpg_photo)\n  saveImageDump = generateEngine_L2.postProcessI2I(f"{jpg_photo}")\n\n'

In [ ]:
#projectDaredevilMerchGen_L2.generateTshirt("1girl, person, long haired girl, disappointed, failed experiment, Test Laboratory, shattered test tube, exploding beaker glass")
#projectDaredevilMerchGen_L2.generateTshirt("1girl, person, long haired girl, Happy, Successful experiment, Green cloud, Test Laboratory, shattered test tube, exploding beaker glass")
#projectDaredevilMerchGen_L2.generateTshirt("Space, Astronauts, Smiling, Saturn, cogwheel planet rings, planets revolves around saturn, starry sky, shooting stars, Flying railways, space train, butterflies, flying whales, Morning rays, aesthethic black golden tea cafe")

## Quick and Dirty YOLO Focus tiling + Matrix Tiling development
#### Tech for version 0.7.3

In [ ]:
def objectDetectionAndRegionOptimization():
  import cv2
  import os
  from urllib import request as wget
  #Change this into Image to Text/Prompt approach (Using YOLO isn't a great idea since it doesn't represent the whole style of the image in texts)
  #Detect objects with YOLO then compare its context using GPT

  # Download YOLOv4 files if not present
  if not os.path.exists("yolov4.weights"):
      wget.urlretrieve("https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights", "yolov4.weights")

  if not os.path.exists("yolov4.cfg"):
      wget.urlretrieve("https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg", "yolov4.cfg")

  if not os.path.exists("coco.names"):
      wget.urlretrieve("https://raw.githubusercontent.com/AlexeyAB/darknet/master/data/coco.names", "coco.names")

  # Load YOLOv4 model
  net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")

  # Load class names
  with open("coco.names", "r") as f:
      classes = [line.strip() for line in f.readlines()]

  # Load image
  #img_url = "https://thumbs.dreamstime.com/z/urban-image-city-road-group-young-people-walking-crossing-zebra-greek-street-background-thessaloniki-greece-march-130858431.jpg" #test image
  #req = wget.urlopen(img_url)
  #arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
  #img = cv2.imdecode(arr, -1)
  image = cv2.imread("/tmp/before_processed.jpg")

  # Detect objects in image
  blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
  net.setInput(blob)
  layer_names = net.getLayerNames()
  output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
  outs = net.forward(output_layers)

  # Render the bounding boxes
  boxes = []
  confidences = []
  class_ids = []
  for out in outs:
      for detection in out:
          scores = detection[5:]
          class_id = np.argmax(scores)
          confidence = scores[class_id]
          if confidence > 0.5:
              center_x = int(detection[0] * image.shape[1])
              center_y = int(detection[1] * image.shape[0])
              width = int(detection[2] * image.shape[1])
              height = int(detection[3] * image.shape[0])
              x = int(center_x - width / 2)
              y = int(center_y - height / 2)
              boxes.append([x, y, width, height])
              confidences.append(float(confidence))
              class_ids.append(class_id)

  # Apply non-maximum suppression to remove overlapping bounding boxes
  indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
  for i in indices:
      i = i[0]
      x, y, w, h = boxes[i]
      label = classes[class_ids[i]]
      cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
      cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

  # Display the image with bounding boxes in Jupyter Notebook
  cv2.imshow('Object Detection', image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

  # Save the result as a static image
  cv2.imwrite('output_image.jpg', image)

#objectDetectionAndRegionOptimization()
#enhanced -> crop focus to the Specific object detected by YOLO -> resize to highest resolution possible reqOptimizedRes -> generate prompt with the detected object -> slap it into the the main Canvas -> PatchGAN -> ProcessFX as usual

# Shell

In [ ]:
#!sh